## 概要

Open-Sora 2.0は、3000万円で実現可能な商用レベルの動画生成モデル

訓練コストは、同等のモデル（MovieGenやStep-Video-T2V）よりも5~10倍低い

人の評価とVBenchのスコアでは、Huyyuan VideoやRunway Gen-3 Alphaに匹敵:

![](image/fig1.png)

- Visual Quality: 視覚品質
- Prompt Following: 指示追従性
- Motion Quality: 動きの品質

## データ

目的は、学習の進捗に合わせたデータピラミッド（hierarchical data pyramid）の構築

様々な種類のデータを検出可能ななフィルタを開発

学習の進捗に応じて、フィルタリングの強度を高め、純度と品質の高い小さいサブセットで訓練

データフィルタリングの全体像:

![](image/fig2.png)

- 紫: 生の動画の前処理
    1. フィルタリング
        - 破損したファイルの除去
        - 極端な動画の除去
            - 再生時間が2秒未満
            - 1画像あたりのデータ量（Bit per pixel）が0.02未満
            - フレームレート（fps）が16未満
            - アスペクト比が範囲外（1/3, 3）
            - 特定の低品質なエンコード設定（Constrained Baseline profile）
    2. 連続した映像を検出し、短いクリップに分割
        - FFmpegのlibavfilterを使用し、シーンスコア（フレーム間の視覚差分）を計算
    3. 動画のフォーマット
        - フレームレートは30fps以下
        - 長辺は1080px以下
        - コーデック（圧縮形式）はH.264
        - 動画の黒帯を削除
        - 8秒を超えるショットは、8秒のクリップに分割し、2秒未満は破棄
- 青: クリップ動画のスコアフィルタリング
    - 美的スコアでのフィルタリング
        - [CLIPとMLP][1]で美的スコアを予測
            - 最初・中間・最終フレームを抽出し、スコアを計算し、平均
    - 鮮明さの低い動画のフィルタリング
        - OpenCVのラプラシアン演算子で画像の分散が低い（ぼやけている）動画を除去
    - 手ブレの多い動画のフィルタリング
        - PySceneDetectを使用して、フレーム間の変化が大きい動画を除去
    - 重複クリップのフィルタリング
- 緑: 256pxの低解像度動画
    - 多くの文章が含まれる動画のフィルタリング
        - PaddleOCRを使用して、テキストのバウンディングボックスを検出
        - 信頼度スコアが0.7を超えるボックスの総面積が多い動画を除去
    - モーションスコアでのフィルタリング
        - libavfilterのVMAFを使用して動画の動きの激しさを測定
        - モーションスコアが極端に低い・高い動画を除去
- 黄色: 768pxの高解像度動画

[1]: https://github.com/christophschuhmann/improved-aesthetic-predictor

動画のキャプションを作成するために、視覚言語モデルを使用:

- 256px動画には、LLaVA-Video
- 778px動画には、Qwen 2.5 Max（ハルシネーションが少ないプロプライエタリモデル）

キャプション生成のプロンプトの構成:

- 主な被写体
- 被写体の動き
- 背景や環境
- 証明条件や雰囲気
- カメラワーク
- リアル・シネマティック・3D・アニメなどの動画のスタイル

生成時に動きの強度を調整可能にするため、キャプションの最後にモーションスコアを追記

フィルタリング後のデータの統計:

![](image/fig3.png)

- 美的スコアは4.5~5.5で中程度
- 動画の長さは2~8秒で、半分近くが6~8秒
- アスペクト比は、大部分が0.5~0.75（16:9の横長動画）
- キャプションの70%は75単語を超えていて情報量が多い

キャプションのワードクラウド:

![](image/fig4.png)

- 背景や照明条件も含まれていて、被写体は人物が多い

## アーキテクチャ

### 3次元オートエンコーダ

[Hunyuan Video VAE][1]のアーキテクチャを効率化したVideo DC-AEを開発

DC-AEは、[Deep Compression Autoencoder][2]の略

圧縮率は、$4\times 32\times 32$（時間は $\frac{1}{4}$、縦と横は$\frac{1}{32}$に圧縮）

オートエンコーダーの学習データは32フレーム、256pxのため、潜在表現は$8\times 8\times 8$

[1]: https://arxiv.org/abs/2412.03603
[2]: https://arxiv.org/abs/2410.10733

Video DC-AEのアーキテクチャ:

![](image/fig5.png)

- エンコーダ
    - 3層のResBlockと3層のEfficientViT Blockで構成される
    - 最初の5つのブロックは、ダウンサンプリング用
    - 学習を可能にするため、ダウンサンプルブロックには、残差接続が導入
    - 残差接続はピクセルアンシャッフリングを使用（Space&Time->Channel）
- デコーダ
    - 3層のEfficientViT Blockと3層のResBlockで構成される
    - 最後の5つのブロックは、アップサンプリング用
    - 学習を可能にするため、アップサンプルブロックには、残差接続が導入
    - 残差接続はピクセルシャッフリングを使用（Channel->Space&Time）

Video DC-AEをスクラッチから学習し、再構成品質を評価:

![](image/table1.png)

- LPIPS: 人間の知覚に近い画質評価指標

## DiTアーキテクチャ

離れたフレームや画素同士の関係を効果的に捉えるフルアテンションを採用

動画はVideo DC-AEで圧縮後、パッチサイズ1（=パッチ化無し）でフラット化

Hunyuan Videoのオートエンコーダーを使用する場合は、パッチサイズ2が必要

FLUXの[MMDiT][1]を参考に、デュアルストリームとシングルストリームからなる構造を採用:

![](image/fig6.png)

[1]: https://github.com/black-forest-labs/flux

- デュアルストリームブロックで、動画とテキストが別々に特徴抽出される
- シングルストリームブロックで、特徴を統合する

空間と時間情報を捉えるために、3D RoPEを採用

テキストノエンコードは、2つの事前学習済みモデルを採用:

- T5-XXL: 複雑なテキストの意味を捉える
- CLIP-Large: テキストと視覚概念の整合性を捉える（=指示追従性を高める）

![](image/table2.png)

## 実装

- image.py: 画像のみで学習。
- stage1.py: 256px解像度の動画で学習。
- stage2.py: 768px解像度の動画で学習（シーケンス並列化を使用、デフォルトは4）。
- stage1_i2v.py: 256px解像度でT2V（テキストから動画）とI2V（画像から動画）を学習。
- stage2_i2v.py: 768px解像度でT2VとI2Vを学習。

## 環境構築

In [ ]:
%load_ext autoreload
%autoreload 2
%cd /workspaces/open-sora/Open-Sora

In [ ]:
import logging
import os
import platform
import subprocess

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)

PYTHON_VERSION = platform.python_version()
logger.info(f"Python {PYTHON_VERSION}")

NVIDIA_SMI = subprocess.run("nvidia-smi", capture_output=True, text=True).stdout
logger.info(f"{NVIDIA_SMI}")

In [ ]:
if False:
    %pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

    %pip install \
        accelerate \
        av \
        colossalai \
        ftfy \
        liger-kernel \
        omegaconf \
        mmengine \
        openai \
        pandas \
        pandarallel \
        pyarrow \
        tensorboard \
        wandb \
        --extra-index-url https://download.pytorch.org/whl/cu124

    %pip install flash-attn --no-build-isolation

    %pip install -e . --no-deps

In [ ]:
from dataclasses import dataclass, field
from einops import rearrange
from flash_attn import flash_attn_func as flash_attn_func_v2
from functools import partial
from huggingface_hub import PyTorchModelHubMixin
from inspect import signature
from liger_kernel.ops.rms_norm import LigerRMSNormFunction
from liger_kernel.ops.rope import LigerRopeFunction
from omegaconf import MISSING, OmegaConf
from torch import Tensor, nn
from torch.nn.modules.batchnorm import _BatchNorm
from typing import Any, Callable, Optional, Union, Tuple
import diffusers
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

try:
    from flash_attn_interface import flash_attn_func as flash_attn_func_v3
    SUPPORT_FA3 = True
except:
    SUPPORT_FA3 = False

## DC-AE

### ユーティリティ関数

In [ ]:
def val2list(x: Union[list, tuple, Any], repeat_time=1) -> list:
    """
    値をリストに変換する

    Args:
        x (Union[list, tuple, Any]): 変換する値
        repeat_time (int, optional): xがリストやタプルでない場合の繰り返し回数
    Returns:
        list: 変換後のリスト
    """

    # リストかタプルの場合
    if isinstance(x, (list, tuple)):
        return list(x)

    # それ以外の場合
    return [x for _ in range(repeat_time)]

# 検証
val2list([1,2,3]), val2list((4,5)), val2list(5, repeat_time=3)

In [ ]:
def val2tuple(x: Union[list, tuple, Any], min_len: int = 1, idx_repeat: int = -1) -> tuple:
    """
    値をタプルに変換する

    Args:
        x (Union[list, tuple, Any]): 変換する値
        min_len (int, optional): タプルの最小長さ
        idx_repeat (int, optional): 繰り返し挿入するインデックス
    Returns:
        tuple: 変換後のタプル
    """

    # 値をリストに変換
    x = val2list(x)

    # 必要に応じて要素を繰り返し
    if len(x) > 0:
        x[idx_repeat:idx_repeat] = [
            x[idx_repeat] for _ in range(min_len - len(x))
        ]

    return tuple(x)

# 検証
val2tuple(False, 2), val2tuple((None, "bnb2d"), 2), val2tuple((None, None), 2)

In [ ]:
def build_kwargs_from_config(config: dict, target_func: Callable) -> dict[str, Any]:
    """
    設定から関数のキーワード引数を構築する

    Args:
        config (dict): 設定辞書
        target_func (Callable): 対象の関数
    Returns:
        dict[str, Any]: 関数に渡すキーワード引数
    """
    logger.info(f"build_kwargs_from_config {config=} {target_func=}")

    valid_keys = list(signature(target_func).parameters)
    kwargs = {}
    for key in config:
        if key in valid_keys:
            kwargs[key] = config[key]
    return kwargs

In [ ]:
def get_same_padding(kernel_size: Union[int, tuple[int, ...]]) -> Union[int, tuple[int, ...]]:
    """
    カーネルサイズに基づいて同じパディングを計算する
    ConvLayer, LiteMLAで使用

    Args:
        kernel_size (Union[int, tuple[int, ...]]): カーネルサイズ
    Returns:
        Union[int, tuple[int, ...]]: 同じパディングサイズ
    """
    if isinstance(kernel_size, tuple):
        return tuple([get_same_padding(ks) for ks in kernel_size])
    else:
        assert kernel_size % 2 > 0, "kernel size should be odd number"
        return kernel_size // 2

In [ ]:
def auto_grad_checkpoint(module, *args, **kwargs):
    """
    自動勾配チェックポイント関数
    勾配チェックポイントが有効な場合、チェックポイントを使用してモジュールを実行する
    そうでない場合、通常のフォワードパスを実行する

    Args:
        module: nn.Moduleまたはnn.ModuleのIterable
        *args: モジュールへの引数
        **kwargs: モジュールへのキーワード引数
    Returns:
        モジュールの出力
    """
    logger.info(f"auto_grad_checkpoint {len(args)=} {args[0].shape=} {kwargs=}")
    if getattr(module, "grad_checkpointing", False):
        if not isinstance(module, Iterable):
            return checkpoint(module, *args, use_reentrant=True, **kwargs)
        gc_step = module[0].grad_checkpointing_step
        return checkpoint_sequential(module, gc_step, *args, use_reentrant=False, **kwargs)
    return module(*args, **kwargs)

### モジュールリスト

In [ ]:
class OpSequential(nn.Module):
    """
    複数の操作を順次適用するモジュール
    build_encoder_project_out_block, Encoder
    build_decoder_project_out_block, Decoder
    """
    def __init__(self, op_list: list[Optional[nn.Module]]):
        super().__init__()
        valid_op_list = []
        for op in op_list:
            if op is not None:
                valid_op_list.append(op)
        self.op_list = nn.ModuleList(valid_op_list)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for op in self.op_list:
            x = op(x)
        return x

### 恒等層

In [ ]:
class IdentityLayer(nn.Module):
    """
    恒等層
    build_block, EfficientViTBlockで使用
    """
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x

### 正規化層

In [ ]:
# register normalization function here
REGISTERED_NORM_DICT: dict[str, type] = {
    "bn2d": nn.BatchNorm2d,
    "ln": nn.LayerNorm,
    # "ln2d": LayerNorm2d,
    # "rms2d": RMSNorm2d,
    # "rms3d": RMSNorm3d,
}

In [ ]:
def build_norm(name="bn2d", num_features=None, **kwargs) -> Optional[nn.Module]:
    """
    正規化層を構築する

    Args:
        name (str, optional): 正規化層の名前
        num_features (int, optional): 正規化層の特徴数
        **kwargs: 正規化層の追加引数
    Returns:
        Optional[nn.Module]: 構築された正規化層、またはNone
    """
    logger.info(f"build_norm {name=} {num_features=} {kwargs=}")

    if name in ["ln", "ln2d"]:
        kwargs["normalized_shape"] = num_features
    else:
        kwargs["num_features"] = num_features
    if name in REGISTERED_NORM_DICT:
        norm_cls = REGISTERED_NORM_DICT[name]
        args = build_kwargs_from_config(kwargs, norm_cls)
        return norm_cls(**args)
    else:
        return None

### 活性化関数

In [ ]:
REGISTERED_ACT_DICT: dict[str, type] = {
    "relu": nn.ReLU,
    "relu6": nn.ReLU6,
    "hswish": nn.Hardswish,
    "silu": nn.SiLU,
    "gelu": partial(nn.GELU, approximate="tanh"),
}

In [ ]:
def build_act(name: str, **kwargs) -> Optional[nn.Module]:
    """
    活性化関数を構築する

    Args:
        name (str): 活性化関数の名前
        **kwargs: 活性化関数の追加引数
    Returns:
        Optional[nn.Module]: 構築された活性化関数、またはNone
    """
    logger.info(f"build_act {name=} {kwargs=}")
    if name in REGISTERED_ACT_DICT:
        act_cls = REGISTERED_ACT_DICT[name]
        args = build_kwargs_from_config(kwargs, act_cls)
        return act_cls(**args)
    else:
        return None

### 畳み込み層

In [ ]:
class ChannelChunkConv3d(nn.Conv3d):
    """
    チャンネルをチャンクに分割してConv3dを実行するクラス
    ConvLayerとLiteMLAで使用
    """

    CONV3D_NUMEL_LIMIT = 2**31

    def _get_output_numel(self, input_shape: torch.Size) -> int:
        numel = self.out_channels
        if len(input_shape) == 5:
            numel *= input_shape[0]
        for i, d in enumerate(input_shape[-3:]):
            d_out = math.floor(
                (d + 2 * self.padding[i] - self.dilation[i] * (self.kernel_size[i] - 1) - 1) / self.stride[i] + 1
            )
            numel *= d_out
        return numel

    def _get_n_chunks(self, numel: int, n_channels: int):
        n_chunks = math.ceil(numel / ChannelChunkConv3d.CONV3D_NUMEL_LIMIT)
        n_chunks = ceil_to_divisible(n_chunks, n_channels)
        return n_chunks

    def forward(self, input: Tensor) -> Tensor:
        """
        入力テンソルに対してチャンク化されたConv3dを実行する
        """
        logger.info(f"ChannelChunkConv3d.forward {input.shape=}")
        if input.numel() // input.size(0) < ChannelChunkConv3d.CONV3D_NUMEL_LIMIT:
            return super().forward(input)
        n_in_chunks = self._get_n_chunks(input.numel(), self.in_channels)
        n_out_chunks = self._get_n_chunks(self._get_output_numel(input.shape), self.out_channels)
        if n_in_chunks == 1 and n_out_chunks == 1:
            return super().forward(input)
        outputs = []
        input_shards = input.chunk(n_in_chunks, dim=1)
        for weight, bias in zip(self.weight.chunk(n_out_chunks), self.bias.chunk(n_out_chunks)):
            weight_shards = weight.chunk(n_in_chunks, dim=1)
            o = None
            for x, w in zip(input_shards, weight_shards):
                if o is None:
                    o = F.conv3d(x, w, bias, self.stride, self.padding, self.dilation, self.groups)
                else:
                    o += F.conv3d(x, w, None, self.stride, self.padding, self.dilation, self.groups)
            outputs.append(o)
        return torch.cat(outputs, dim=1)

In [ ]:
class ConvLayer(nn.Module):
    """
    2Dまたは3D畳み込み層、正規化、活性化関数を組み合わせたモジュール
    - build_downsample_block
    - build_encoder_project_in_block
    - GLUMBConv
    - InterpolateConvUpSamplerLayer
    - LiteMLA
    - ResBlock
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size=3, stride=1, dilation=1, groups=1, use_bias=False, dropout=0, norm="bn2d", act_func="relu", is_video=False, pad_mode_3d="constant"):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            kernel_size (int, optional): 畳み込みカーネルのサイズ
            stride (int, optional): 畳み込みのストライド
            dilation (int, optional): 畳み込みのダイレーション
            groups (int, optional): 畳み込みのグループ数
            use_bias (bool, optional): バイアスを使用するかどうか
            dropout (float, optional): ドロップアウト率
            norm (str, optional): 正規化層の種類
            act_func (str, optional): 活性化関数の種類
            is_video (bool, optional): 3D畳み込みを使用するかどうか
            pad_mode_3d (str, optional): 3D畳み込みのパディングモード
        """
        logger.info(f"ConvLayer.__init__ {in_channels=} {out_channels=} {kernel_size=} {stride=} {dilation=} {groups=} {use_bias=} {dropout=} {norm=} {act_func=} {is_video=} {pad_mode_3d=}")

        super().__init__()
        self.is_video = is_video

        if self.is_video:
            assert dilation == 1, "only support dilation=1 for 3d conv"
            assert kernel_size % 2 == 1, "only support odd kernel size for 3d conv"
            self.pad_mode_3d = pad_mode_3d  # 3d padding follows CausalConv3d by Hunyuan
            # padding = (
            #     kernel_size // 2,
            #     kernel_size // 2,
            #     kernel_size // 2,
            #     kernel_size // 2,
            #     kernel_size - 1,
            #     0,
            # )  # W, H, T
            # non-causal padding
            padding = (
                kernel_size // 2,
                kernel_size // 2,
                kernel_size // 2,
                kernel_size // 2,
                kernel_size // 2,
                kernel_size // 2,
            )
            self.padding = padding
            self.dropout = nn.Dropout3d(dropout, inplace=False) if dropout > 0 else None
            assert isinstance(stride, (int, tuple)), "stride must be an integer or 3-tuple for 3d conv"
            self.conv = ChannelChunkConv3d(  # padding is handled by F.pad() in forward()
                in_channels,
                out_channels,
                kernel_size=(kernel_size, kernel_size, kernel_size),
                stride=(stride, stride, stride) if isinstance(stride, int) else stride,
                groups=groups,
                bias=use_bias,
            )
        else:
            padding = get_same_padding(kernel_size)
            padding *= dilation
            self.dropout = nn.Dropout2d(dropout, inplace=False) if dropout > 0 else None
            self.conv = nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=(kernel_size, kernel_size),
                stride=(stride, stride),
                padding=padding,
                dilation=(dilation, dilation),
                groups=groups,
                bias=use_bias,
            )

        self.norm = build_norm(norm, num_features=out_channels)
        self.act = build_act(act_func)
        self.pad = F.pad

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            torch.Tensor: 出力テンソル
        """
        logger.info(f"ConvLayer.forward {x.shape=}")
        if self.dropout is not None:
            x = self.dropout(x)
        if self.is_video:  # custom padding for 3d conv
            x = self.pad(x, self.padding, mode=self.pad_mode_3d)  # "constant" padding defaults to 0
        x = self.conv(x)
        if self.norm:
            x = self.norm(x)
        if self.act:
            x = self.act(x)
        return x

In [ ]:
class GLUMBConv(nn.Module):
    """
    GLUを用いたMBConvブロック
    EfficientViTBlockで使用
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size=3, stride=1, mid_channels=None, expand_ratio=6, use_bias=False, norm=(None, None, "ln2d"), act_func=("silu", "silu", None), is_video=False):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            kernel_size (int, optional): 畳み込みカーネルのサイズ
            stride (int, optional): 畳み込みのストライド
            mid_channels (int, optional): 中間チャネル数
            expand_ratio (float, optional): 拡張比率
            use_bias (bool or tuple, optional): バイアスを使用するかどうか
            norm (str or tuple, optional): 正規化層の種類
            act_func (str or tuple, optional): 活性化関数の種類
            is_video (bool, optional): 3D畳み込みを使用するかどうか
        """
        logger.info(f"GLUMBConv.__init__ {in_channels=} {out_channels=} {kernel_size=} {stride=} {mid_channels=} {expand_ratio=} {use_bias=} {norm=} {act_func=} {is_video=}")

        super().__init__()
        use_bias = val2tuple(use_bias, 3)
        norm = val2tuple(norm, 3)
        act_func = val2tuple(act_func, 3)

        mid_channels = round(in_channels * expand_ratio) if mid_channels is None else mid_channels

        self.glu_act = build_act(act_func[1], inplace=False)
        self.inverted_conv = ConvLayer(
            in_channels,
            mid_channels * 2,
            1,
            use_bias=use_bias[0],
            norm=norm[0],
            act_func=act_func[0],
            is_video=is_video,
        )
        self.depth_conv = ConvLayer(
            mid_channels * 2,
            mid_channels * 2,
            kernel_size,
            stride=stride,
            groups=mid_channels * 2,
            use_bias=use_bias[1],
            norm=norm[1],
            act_func=None,
            is_video=is_video,
        )
        self.point_conv = ConvLayer(
            mid_channels,
            out_channels,
            1,
            use_bias=use_bias[2],
            norm=norm[2],
            act_func=act_func[2],
            is_video=is_video,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"GLUMBConv.forward {x.shape=}")
        x = self.inverted_conv(x)
        x = self.depth_conv(x)

        x, gate = torch.chunk(x, 2, dim=1)
        gate = self.glu_act(gate)
        x = x * gate

        x = self.point_conv(x)
        return x

### ピクセルシャッフリング

In [ ]:
class PixelUnshuffleChannelAveragingDownSampleLayer(nn.Module):
    """
    ピクセルアンシャッフルとチャネル平均化によるダウンサンプリング層
    - build_downsample_block
    - build_encoder_project_out_block
    """

    def __init__(self, in_channels: int, out_channels: int, factor: int, temporal_downsample: bool = False):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            factor (int): ダウンサンプリングファクター
            temporal_downsample (bool, optional):
                5D入力テンソルに対して時間方向のダウンサンプリングを行うかどうか
        """
        logger.info(f"PixelUnshuffleChannelAveragingDownSampleLayer.__init__ {in_channels=} {out_channels=} {factor=} {temporal_downsample=}")

        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.factor = factor
        self.temporal_downsample = temporal_downsample

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"PixelUnshuffleChannelAveragingDownSampleLayer.forward {x.shape=}")

        if x.dim() == 4:
            assert self.in_channels * self.factor**2 % self.out_channels == 0
            group_size = self.in_channels * self.factor**2 // self.out_channels
            x = F.pixel_unshuffle(x, self.factor)
            B, C, H, W = x.shape
            x = x.view(B, self.out_channels, group_size, H, W)
            x = x.mean(dim=2)
        elif x.dim() == 5:  # [B, C, T, H, W]
            _, _, T, _, _ = x.shape
            if self.temporal_downsample and T != 1:  # 3d pixel unshuffle
                x = pixel_unshuffle_3d(x, self.factor)
                assert self.in_channels * self.factor**3 % self.out_channels == 0
                group_size = self.in_channels * self.factor**3 // self.out_channels
            else:  # 2d pixel unshuffle
                x = x.permute(0, 2, 1, 3, 4)  # [B, T, C, H, W]
                x = F.pixel_unshuffle(x, self.factor)
                x = x.permute(0, 2, 1, 3, 4)  # [B, C, T, H, W]
                assert self.in_channels * self.factor**2 % self.out_channels == 0
                group_size = self.in_channels * self.factor**2 // self.out_channels
            B, C, T, H, W = x.shape
            x = x.view(B, self.out_channels, group_size, T, H, W)
            x = x.mean(dim=2)
        else:
            raise ValueError(f"Unsupported input dimension: {x.dim()}")
        return x

    def __repr__(self):
        return f"PixelUnshuffleChannelAveragingDownSampleLayer(in_channels={self.in_channels}, out_channels={self.out_channels}, factor={self.factor}), temporal_downsample={self.temporal_downsample}"


In [ ]:
class ChannelDuplicatingPixelShuffleUpSampleLayer(nn.Module):
    """
    チャンネル複製とピクセルシャッフルによるアップサンプリング層
    build_upsample_block, build_decoder_project_in_blockで使用
    """

    def __init__(self, in_channels: int, out_channels: int, factor: int, temporal_upsample: bool = False):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            factor (int): アップサンプリングファクター
            temporal_upsample (bool, optional):
                5D入力テンソルに対して時間方向のアップサンプリングを行うかどうか
        """
        logger.info(f"ChannelDuplicatingPixelShuffleUpSampleLayer.__init__ {in_channels=} {out_channels=} {factor=} {temporal_upsample=}")
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.factor = factor
        assert out_channels * factor**2 % in_channels == 0
        self.temporal_upsample = temporal_upsample

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"ChannelDuplicatingPixelShuffleUpSampleLayer.forward {x.shape=}")
        if x.dim() == 5:
            B, C, T, H, W = x.shape
            assert C == self.in_channels

        if self.temporal_upsample and T != 1:  # video input
            repeats = self.out_channels * self.factor**3 // self.in_channels
        else:
            repeats = self.out_channels * self.factor**2 // self.in_channels

        x = x.repeat_interleave(repeats, dim=1)

        if x.dim() == 4:  # original image-only training
            x = F.pixel_shuffle(x, self.factor)
        elif x.dim() == 5:  # [B, C, T, H, W]
            if self.temporal_upsample and T != 1:  # video input
                x = pixel_shuffle_3d(x, self.factor)
            else:
                x = x.permute(0, 2, 1, 3, 4)  # [B, T, C, H, W]
                x = F.pixel_shuffle(x, self.factor)  # on H and W only
                x = x.permute(0, 2, 1, 3, 4)  # [B, C, T, H, W]
        return x

    def __repr__(self):
        return f"ChannelDuplicatingPixelShuffleUpSampleLayer(in_channels={self.in_channels}, out_channels={self.out_channels}, factor={self.factor}, temporal_upsample={self.temporal_upsample})"

### InterpolateConvUpSamplerLayer

In [ ]:

VERBOSE = False

def chunked_interpolate(x, scale_factor, mode="nearest"):
    """
    チャンネルの次元に沿ってチャンク化して大きなテンソルを補間する
    現在は「nearest」補間モードのみサポート

    https://discuss.pytorch.org/t/error-using-f-interpolate-for-large-3d-input/207859

    Args:
        x (torch.Tensor): 入力テンソル (B, C, D, H, W)
        scale_factor: スケーリングファクターのタプル (d, h, w)
    Returns:
        torch.Tensor: 補間されたテンソル
    """
    logger.info(f"chunked_interpolate {x.shape=} {scale_factor=} {mode=}")

    assert (
        mode == "nearest"
    ), "Only the nearest mode is supported"  # actually other modes are theoretically supported but not tested
    if len(x.shape) != 5:
        raise ValueError("Expected 5D input tensor (B, C, D, H, W)")

    # Calculate max chunk size to avoid int32 overflow. num_elements < max_int32
    # Max int32 is 2^31 - 1
    max_elements_per_chunk = 2**31 - 1

    # Calculate output spatial dimensions
    out_d = math.ceil(x.shape[2] * scale_factor[0])
    out_h = math.ceil(x.shape[3] * scale_factor[1])
    out_w = math.ceil(x.shape[4] * scale_factor[2])

    # Calculate max channels per chunk to stay under limit
    elements_per_channel = out_d * out_h * out_w
    max_channels = max_elements_per_chunk // (x.shape[0] * elements_per_channel)

    # Use smaller of max channels or input channels
    chunk_size = min(max_channels, x.shape[1])

    # Ensure at least 1 channel per chunk
    chunk_size = max(1, chunk_size)
    if VERBOSE:
        print(f"Input channels: {x.shape[1]}")
        print(f"Chunk size: {chunk_size}")
        print(f"max_channels: {max_channels}")
        print(f"num_chunks: {math.ceil(x.shape[1] / chunk_size)}")

    chunks = []
    for i in range(0, x.shape[1], chunk_size):
        start_idx = i
        end_idx = min(i + chunk_size, x.shape[1])

        chunk = x[:, start_idx:end_idx, :, :, :]

        interpolated_chunk = F.interpolate(chunk, scale_factor=scale_factor, mode="nearest")

        chunks.append(interpolated_chunk)

    if not chunks:
        raise ValueError(f"No chunks were generated. Input shape: {x.shape}")

    # Concatenate chunks along channel dimension
    return torch.cat(chunks, dim=1)

class InterpolateConvUpSampleLayer(nn.Module):
    """
    補間と畳み込みを組み合わせたアップサンプリング層
    build_upsample_blockで使用
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, factor: int, mode: str = "nearest", is_video: bool = False, temporal_upsample: bool = False) -> None:
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            kernel_size (int): 畳み込みカーネルのサイズ
            factor (int): アップサンプリングファクター
            mode (str, optional): 補間モード
            is_video (bool, optional): 3D畳み込みを使用するかどうか
            temporal_upsample (bool, optional):
                5D入力テンソルに対して時間方向のアップサンプリングを行うかどうか
        """
        super().__init__()
        self.factor = factor
        self.mode = mode
        self.temporal_upsample = temporal_upsample
        self.conv = ConvLayer(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            use_bias=True,
            norm=None,
            act_func=None,
            is_video=is_video,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"InterpolateConvUpSampleLayer.forward {x.shape=}")
        if x.dim() == 4:
            x = F.interpolate(x, scale_factor=self.factor, mode=self.mode)
        elif x.dim() == 5:
            # [B, C, T, H, W] -> [B, C, T*factor, H*factor, W*factor]
            if self.temporal_upsample and x.size(2) != 1:  # temporal upsample for video input
                x = chunked_interpolate(x, scale_factor=[self.factor, self.factor, self.factor], mode=self.mode)
            else:
                x = chunked_interpolate(x, scale_factor=[1, self.factor, self.factor], mode=self.mode)
        x = self.conv(x)
        return x

    def __repr__(self):
        return f"InterpolateConvUpSampleLayer(factor={self.factor}, mode={self.mode}, temporal_upsample={self.temporal_upsample})"

### アテンション機構

In [ ]:
class LiteMLA(nn.Module):
    """
    軽量なマルチスケール線形アテンション機構
    EfficientViTBlockで使用
    """

    def __init__(self, in_channels: int, out_channels: int, heads: Optional[int] = None, heads_ratio: float = 1.0, dim=8, use_bias=False, norm=(None, "bn2d"), act_func=(None, None), kernel_func="relu", scales: tuple[int, ...] = (5,), eps=1.0e-15, is_video=False):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            heads (Optional[int], optional): アテンションヘッド数
            heads_ratio (float, optional): ヘッド数の比率
            dim (int, optional): 各ヘッドの次元数
            use_bias (bool or tuple, optional): バイアスを使用するかどうか
            norm (str or tuple, optional): 正規化層の種類
            act_func (str or tuple, optional): 活性化関数の種類
            kernel_func (str, optional): カーネル関数の種類
            scales (tuple[int, ...], optional): マルチスケール畳み込みのスケール
            eps (float, optional): 数値安定性のための小さな値
            is_video (bool, optional): 3D畳み込みを使用するかどうか
        """
        logger.info(f"LiteMLA.__init__ {in_channels=} {out_channels=} {heads=} {heads_ratio=} {dim=} {use_bias=} {norm=} {act_func=} {kernel_func=} {scales=} {eps=} {is_video=}")

        super().__init__()
        self.eps = eps
        heads = int(in_channels // dim * heads_ratio) if heads is None else heads

        total_dim = heads * dim

        use_bias = val2tuple(use_bias, 2)
        norm = val2tuple(norm, 2)
        act_func = val2tuple(act_func, 2)

        self.dim = dim
        self.qkv = ConvLayer(
            in_channels,
            3 * total_dim,
            1,
            use_bias=use_bias[0],
            norm=norm[0],
            act_func=act_func[0],
            is_video=is_video,
        )
        conv_class = nn.Conv2d if not is_video else ChannelChunkConv3d
        self.aggreg = nn.ModuleList(
            [
                nn.Sequential(
                    conv_class(
                        3 * total_dim,
                        3 * total_dim,
                        scale,
                        padding=get_same_padding(scale),
                        groups=3 * total_dim,
                        bias=use_bias[0],
                    ),
                    conv_class(3 * total_dim, 3 * total_dim, 1, groups=3 * heads, bias=use_bias[0]),
                )
                for scale in scales
            ]
        )
        self.kernel_func = build_act(kernel_func, inplace=False)

        self.proj = ConvLayer(
            total_dim * (1 + len(scales)),
            out_channels,
            1,
            use_bias=use_bias[1],
            norm=norm[1],
            act_func=act_func[1],
            is_video=is_video,
        )

    @torch.autocast(device_type="cuda", enabled=False)
    def relu_linear_att(self, qkv: torch.Tensor) -> torch.Tensor:
        """
        ReLUカーネルを用いた線形アテンション
        """
        logger.info(f"LiteMLA.relu_linear_att {qkv.shape=}")

        if qkv.ndim == 5:
            B, _, T, H, W = list(qkv.size())
            is_video = True
        else:
            B, _, H, W = list(qkv.size())
            is_video = False

        if qkv.dtype == torch.float16:
            qkv = qkv.float()

        if qkv.ndim == 4:
            qkv = torch.reshape(
                qkv,
                (
                    B,
                    -1,
                    3 * self.dim,
                    H * W,
                ),
            )
        elif qkv.ndim == 5:
            qkv = torch.reshape(
                qkv,
                (
                    B,
                    -1,
                    3 * self.dim,
                    H * W * T,
                ),
            )
        q, k, v = (
            qkv[:, :, 0 : self.dim],
            qkv[:, :, self.dim : 2 * self.dim],
            qkv[:, :, 2 * self.dim :],
        )

        # lightweight linear attention
        q = self.kernel_func(q)
        k = self.kernel_func(k)

        # linear matmul
        trans_k = k.transpose(-1, -2)

        v = F.pad(v, (0, 0, 0, 1), mode="constant", value=1)
        vk = torch.matmul(v, trans_k)
        out = torch.matmul(vk, q)
        if out.dtype == torch.bfloat16:
            out = out.float()
        out = out[:, :, :-1] / (out[:, :, -1:] + self.eps)

        if not is_video:
            out = torch.reshape(out, (B, -1, H, W))
        else:
            out = torch.reshape(out, (B, -1, T, H, W))
        return out

    @torch.autocast(device_type="cuda", enabled=False)
    def relu_quadratic_att(self, qkv: torch.Tensor) -> torch.Tensor:
        """
        Quadratic Attention with ReLU kernel
        今回は使用しない
        """
        B, _, H, W = list(qkv.size())

        qkv = torch.reshape(
            qkv,
            (
                B,
                -1,
                3 * self.dim,
                H * W,
            ),
        )
        q, k, v = (
            qkv[:, :, 0 : self.dim],
            qkv[:, :, self.dim : 2 * self.dim],
            qkv[:, :, 2 * self.dim :],
        )

        q = self.kernel_func(q)
        k = self.kernel_func(k)

        att_map = torch.matmul(k.transpose(-1, -2), q)  # b h n n
        original_dtype = att_map.dtype
        if original_dtype in [torch.float16, torch.bfloat16]:
            att_map = att_map.float()
        att_map = att_map / (torch.sum(att_map, dim=2, keepdim=True) + self.eps)  # b h n n
        att_map = att_map.to(original_dtype)
        out = torch.matmul(v, att_map)  # b h d n

        out = torch.reshape(out, (B, -1, H, W))
        return out

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"LiteMLA.forward {x.shape=}")
        # generate multi-scale q, k, v
        qkv = self.qkv(x)
        multi_scale_qkv = [qkv]
        for op in self.aggreg:
            multi_scale_qkv.append(op(qkv))
        qkv = torch.cat(multi_scale_qkv, dim=1)

        if qkv.ndim == 4:
            H, W = list(qkv.size())[-2:]
            # num_tokens = H * W
        elif qkv.ndim == 5:
            _, _, T, H, W = list(qkv.size())
            # num_tokens = H * W * T

        # if num_tokens > self.dim:
        out = self.relu_linear_att(qkv).to(qkv.dtype)
        # else:
        #     if self.is_video:
        #         raise NotImplementedError("Video is not supported for quadratic attention")
        #     out = self.relu_quadratic_att(qkv)
        out = self.proj(out)

        return out

### ResidualBlock

In [ ]:
class ResidualBlock(nn.Module):
    """
    残差接続を持つブロック
    EfficientViTBlockで使用
    """

    def __init__(
        self,
        main: Optional[nn.Module],
        shortcut: Optional[nn.Module],
        post_act=None,
        pre_norm: Optional[nn.Module] = None,
    ):
        super().__init__()

        self.pre_norm = pre_norm
        self.main = main
        self.shortcut = shortcut
        self.post_act = build_act(post_act)

    def forward_main(self, x: torch.Tensor) -> torch.Tensor:
        if self.pre_norm is None:
            return self.main(x)
        else:
            return self.main(self.pre_norm(x))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.main is None:
            res = x
        elif self.shortcut is None:
            res = self.forward_main(x)
        else:
            res = self.forward_main(x) + self.shortcut(x)
            if self.post_act:
                res = self.post_act(res)
        return res

### EfficientViT

In [ ]:
class EfficientViTBlock(nn.Module):
    """
    EfficientViTブロック
    build_blockで使用
    """
    def __init__(self, in_channels: int, heads_ratio: float = 1.0, dim=32, expand_ratio: float = 4, scales: tuple[int, ...] = (5,), norm: str = "bn2d", act_func: str = "hswish", context_module: str = "LiteMLA", local_module: str = "MBConv", is_video: bool = False):
        """
        Args:
            in_channels (int): 入力チャネル数
            heads_ratio (float, optional): ヘッド数の比率
            dim (int, optional): 各ヘッドの次元数
            expand_ratio (float, optional): 拡張比率
            scales (tuple[int, ...], optional): マルチスケール畳み込みのスケール
            norm (str, optional): 正規化層の種類
            act_func (str, optional): 活性化関数の種類
            context_module (str, optional): コンテキストモジュールの種類
            local_module (str, optional): ローカルモジュールの種類
            is_video (bool, optional): 3D畳み込みを使用するかどうか
        """
        logger.info(f"EfficientViTBlock.__init__ {in_channels=} {heads_ratio=} {dim=} {expand_ratio=} {scales=} {norm=} {act_func=} {context_module=} {local_module=} {is_video=}")

        super().__init__()
        if context_module == "LiteMLA":
            self.context_module = ResidualBlock(
                LiteMLA(
                    in_channels=in_channels,
                    out_channels=in_channels,
                    heads_ratio=heads_ratio,
                    dim=dim,
                    norm=(None, norm),
                    scales=scales,
                    is_video=is_video,
                ),
                IdentityLayer(),
            )
        else:
            raise ValueError(f"context_module {context_module} is not supported")
        if local_module == "MBConv":
            self.local_module = ResidualBlock(
                MBConv(
                    in_channels=in_channels,
                    out_channels=in_channels,
                    expand_ratio=expand_ratio,
                    use_bias=(True, True, False),
                    norm=(None, None, norm),
                    act_func=(act_func, act_func, None),
                    is_video=is_video,
                ),
                IdentityLayer(),
            )
        elif local_module == "GLUMBConv":
            self.local_module = ResidualBlock(
                GLUMBConv(
                    in_channels=in_channels,
                    out_channels=in_channels,
                    expand_ratio=expand_ratio,
                    use_bias=(True, True, False),
                    norm=(None, None, norm),
                    act_func=(act_func, act_func, None),
                    is_video=is_video,
                ),
                IdentityLayer(),
            )
        else:
            raise NotImplementedError(f"local_module {local_module} is not supported")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"EfficientViTBlock.forward {x.shape=}")
        x = self.context_module(x)
        x = self.local_module(x)
        return x

### ResBlock

In [ ]:
class ResBlock(nn.Module):
    """
    残差ブロック
    build_blockで使用
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size=3,
        stride=1,
        mid_channels=None,
        expand_ratio=1,
        use_bias=False,
        norm=("bn2d", "bn2d"),
        act_func=("relu6", None),
        is_video=False,
    ):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            kernel_size (int, optional): 畳み込みカーネルのサイズ
            stride (int, optional): 畳み込みのストライド
            mid_channels (int, optional): 中間チャネル数
            expand_ratio (float, optional): 拡張比率
            use_bias (bool or tuple, optional): バイアスを使用するかどうか
            norm (str or tuple, optional): 正規化層の種類
            act_func (str or tuple, optional): 活性化関数の種類
            is_video (bool, optional): 3D畳み込みを使用するかどうか
        """
        logger.info(f"ResBlock.__init__ {in_channels=} {out_channels=} {kernel_size=} {stride=} {mid_channels=} {expand_ratio=} {use_bias=} {norm=} {act_func=} {is_video=}")

        super().__init__()
        use_bias = val2tuple(use_bias, 2)
        norm = val2tuple(norm, 2)
        act_func = val2tuple(act_func, 2)

        mid_channels = round(in_channels * expand_ratio) if mid_channels is None else mid_channels

        self.conv1 = ConvLayer(
            in_channels,
            mid_channels,
            kernel_size,
            stride,
            use_bias=use_bias[0],
            norm=norm[0],
            act_func=act_func[0],
            is_video=is_video,
        )
        self.conv2 = ConvLayer(
            mid_channels,
            out_channels,
            kernel_size,
            1,
            use_bias=use_bias[1],
            norm=norm[1],
            act_func=act_func[1],
            is_video=is_video,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"ResBlock.forward {x.shape=}")
        x = self.conv1(x)
        x = self.conv2(x)
        return x

### Encoder

In [ ]:
def build_downsample_block(
    block_type: str,
    in_channels: int,
    out_channels: int,
    shortcut: Optional[str],
    is_video: bool,
    temporal_downsample: bool = False,
) -> nn.Module:
    """
    ダウンサンプリングブロックを構築する
    空間的なダウンサンプリングは常に行われる
    時間的なダウンサンプリングはオプション
    build_encoder_project_in_block, Encoderで使用

    Args:
        block_type (str): ブロックの種類 ("Conv" または "ConvPixelUnshuffle")
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        shortcut (Optional[str]): ショートカットの種類 (None または "averaging")
        is_video (bool): 3D畳み込みを使用するかどうか
        temporal_downsample (bool, optional):
            時間方向のダウンサンプリングを行うかどうか
    Returns:
        nn.Module: ダウンサンプリングブロック
    """
    logger.info(f"build_downsample_block {block_type=} {in_channels=} {out_channels=} {shortcut=} {is_video=} {temporal_downsample=}")

    if block_type == "Conv":
        if is_video:
            if temporal_downsample:
                stride = (2, 2, 2)
            else:
                stride = (1, 2, 2)
        else:
            stride = 2
        block = ConvLayer(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=stride,
            use_bias=True,
            norm=None,
            act_func=None,
            is_video=is_video,
        )
    elif block_type == "ConvPixelUnshuffle":
        if is_video:
            raise NotImplementedError("ConvPixelUnshuffle downsample is not supported for video")
        block = ConvPixelUnshuffleDownSampleLayer(
            in_channels=in_channels, out_channels=out_channels, kernel_size=3, factor=2
        )
    else:
        raise ValueError(f"block_type {block_type} is not supported for downsampling")
    if shortcut is None:
        pass
    elif shortcut == "averaging":
        shortcut_block = PixelUnshuffleChannelAveragingDownSampleLayer(
            in_channels=in_channels, out_channels=out_channels, factor=2, temporal_downsample=temporal_downsample
        )
        block = ResidualBlock(block, shortcut_block)
    else:
        raise ValueError(f"shortcut {shortcut} is not supported for downsample")
    return block

In [ ]:
def build_encoder_project_in_block(in_channels: int, out_channels: int, factor: int, downsample_block_type: str, is_video: bool):
    """
    エンコーダの入力プロジェクションブロックを構築する

    Args:
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        factor (int): ダウンサンプリングファクター
        downsample_block_type (str): ダウンサンプリングブロックの種類
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: エンコーダの入力プロジェクションブロック
    """
    logger.info(f"build_encoder_project_in_block {in_channels=} {out_channels=} {factor=} {downsample_block_type=} {is_video=}")

    if factor == 1:
        block = ConvLayer(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=1,
            use_bias=True,
            norm=None,
            act_func=None,
            is_video=is_video,
        )
    elif factor == 2:
        if is_video:
            raise NotImplementedError("Downsample during project_in is not supported for video")
        block = build_downsample_block(
            block_type=downsample_block_type, in_channels=in_channels, out_channels=out_channels, shortcut=None
        )
    else:
        raise ValueError(f"downsample factor {factor} is not supported for encoder project in")
    return block

In [ ]:
def build_encoder_project_out_block(
    in_channels: int,
    out_channels: int,
    norm: Optional[str],
    act: Optional[str],
    shortcut: Optional[str],
    is_video: bool,
):
    """
    エンコーダの出力プロジェクションブロックを構築する
    Encoderで使用

    Args:
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        norm (Optional[str]): 正規化層の種類
        act (Optional[str]): 活性化関数の種類
        shortcut (Optional[str]): ショートカットの種類 (None または "averaging")
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: エンコーダの出力プロジェクションブロック
    """
    logger.info(f"build_encoder_project_out_block {in_channels=} {out_channels=} {norm=} {act=} {shortcut=} {is_video=}")

    block = OpSequential(
        [
            build_norm(norm),
            build_act(act),
            ConvLayer(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=1,
                use_bias=True,
                norm=None,
                act_func=None,
                is_video=is_video,
            ),
        ]
    )
    if shortcut is None:
        pass
    elif shortcut == "averaging":
        shortcut_block = PixelUnshuffleChannelAveragingDownSampleLayer(
            in_channels=in_channels, out_channels=out_channels, factor=1
        )
        block = ResidualBlock(block, shortcut_block)
    else:
        raise ValueError(f"shortcut {shortcut} is not supported for encoder project out")
    return block

In [ ]:
def build_block(
    block_type: str, in_channels: int, out_channels: int, norm: Optional[str], act: Optional[str], is_video: bool
) -> nn.Module:
    """
    ブロックを構築する
    build_stage_mainで使用
    Args:
        block_type (str): ブロックの種類 ("ResBlock", "EViT_GLU", "EViTS5_GLU")
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        norm (Optional[str]): 正規化層の種類
        act (Optional[str]): 活性化関数の種類
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: ブロック
    """
    logger.info(f"build_block {block_type=} {in_channels=} {out_channels=} {norm=} {act=} {is_video=}")

    if block_type == "ResBlock":
        assert in_channels == out_channels
        main_block = ResBlock(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=1,
            use_bias=(True, False),
            norm=(None, norm),
            act_func=(act, None),
            is_video=is_video,
        )
        block = ResidualBlock(main_block, IdentityLayer())
    elif block_type == "EViT_GLU":
        assert in_channels == out_channels
        block = EfficientViTBlock(
            in_channels, norm=norm, act_func=act, local_module="GLUMBConv", scales=(), is_video=is_video
        )
    elif block_type == "EViTS5_GLU":
        assert in_channels == out_channels
        block = EfficientViTBlock(
            in_channels, norm=norm, act_func=act, local_module="GLUMBConv", scales=(5,), is_video=is_video
        )
    else:
        raise ValueError(f"block_type {block_type} is not supported")
    return block

In [ ]:
def build_stage_main(
    width: int, depth: int, block_type: str | list[str], norm: str, act: str, input_width: int, is_video: bool
) -> list[nn.Module]:
    """
    ステージのメインブロックを構築する
    Encoder, Decoderで使用
    Args:
        width (int): ブロックのチャネル数
        depth (int): ブロックの深さ
        block_type (str or list[str]): ブロックの種類
        norm (str): 正規化層の種類
        act (str): 活性化関数の種類
        input_width (int): 入力チャネル数
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        list[nn.Module]: ステージのメインブロックのリスト
    """
    logger.info(f"build_stage_main {width=} {depth=} {block_type=} {norm=} {act=} {input_width=} {is_video=}")

    assert isinstance(block_type, str) or (isinstance(block_type, list) and depth == len(block_type))
    stage = []
    for d in range(depth):
        current_block_type = block_type[d] if isinstance(block_type, list) else block_type
        block = build_block(
            block_type=current_block_type,
            in_channels=width if d > 0 else input_width,
            out_channels=width,
            norm=norm,
            act=act,
            is_video=is_video,
        )
        stage.append(block)
    return stage

In [ ]:
@dataclass
class EncoderConfig:
    in_channels: int = MISSING
    latent_channels: int = MISSING
    width_list: tuple[int, ...] = (128, 256, 512, 512, 1024, 1024)
    depth_list: tuple[int, ...] = (2, 2, 2, 2, 2, 2)
    block_type: Any = "ResBlock"
    norm: str = "rms2d"
    act: str = "silu"
    downsample_block_type: str = "ConvPixelUnshuffle"
    downsample_match_channel: bool = True
    downsample_shortcut: Optional[str] = "averaging"
    out_norm: Optional[str] = None
    out_act: Optional[str] = None
    out_shortcut: Optional[str] = "averaging"
    double_latent: bool = False
    is_video: bool = False
    temporal_downsample: tuple[bool, ...] = ()

In [ ]:
class Encoder(nn.Module):
    """
    エンコーダ
    DCAEで使用
    """

    def __init__(self, cfg: EncoderConfig):
        """
        Args:
            cfg (EncoderConfig): エンコーダの設定
        """
        logger.info(f"Encoder.__init__ {cfg=}")
        super().__init__()
        self.cfg = cfg
        num_stages = len(cfg.width_list)
        self.num_stages = num_stages
        assert len(cfg.depth_list) == num_stages
        assert len(cfg.width_list) == num_stages
        assert isinstance(cfg.block_type, str) or (
            isinstance(cfg.block_type, list) and len(cfg.block_type) == num_stages
        )

        self.project_in = build_encoder_project_in_block(
            in_channels=cfg.in_channels,
            out_channels=cfg.width_list[0] if cfg.depth_list[0] > 0 else cfg.width_list[1],
            factor=1 if cfg.depth_list[0] > 0 else 2,
            downsample_block_type=cfg.downsample_block_type,
            is_video=cfg.is_video,
        )

        self.stages: list[OpSequential] = []
        for stage_id, (width, depth) in enumerate(zip(cfg.width_list, cfg.depth_list)):
            block_type = cfg.block_type[stage_id] if isinstance(cfg.block_type, list) else cfg.block_type
            stage = build_stage_main(
                width=width,
                depth=depth,
                block_type=block_type,
                norm=cfg.norm,
                act=cfg.act,
                input_width=width,
                is_video=cfg.is_video,
            )

            if stage_id < num_stages - 1 and depth > 0:
                downsample_block = build_downsample_block(
                    block_type=cfg.downsample_block_type,
                    in_channels=width,
                    out_channels=cfg.width_list[stage_id + 1] if cfg.downsample_match_channel else width,
                    shortcut=cfg.downsample_shortcut,
                    is_video=cfg.is_video,
                    temporal_downsample=cfg.temporal_downsample[stage_id] if cfg.temporal_downsample != [] else False,
                )
                stage.append(downsample_block)
            self.stages.append(OpSequential(stage))
        self.stages = nn.ModuleList(self.stages)

        self.project_out = build_encoder_project_out_block(
            in_channels=cfg.width_list[-1],
            out_channels=2 * cfg.latent_channels if cfg.double_latent else cfg.latent_channels,
            norm=cfg.out_norm,
            act=cfg.out_act,
            shortcut=cfg.out_shortcut,
            is_video=cfg.is_video,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"Encoder.forward {x.shape=}")
        x = self.project_in(x)
        # x = auto_grad_checkpoint(self.project_in, x)
        for stage in self.stages:
            if len(stage.op_list) == 0:
                continue
            x = auto_grad_checkpoint(stage, x)
        # x = self.project_out(x)
        x = auto_grad_checkpoint(self.project_out, x)
        return x

### Decoder

In [ ]:
def build_decoder_project_in_block(in_channels: int, out_channels: int, shortcut: Optional[str], is_video: bool):
    """
    デコーダの入力プロジェクションブロックを構築する
    Decoderで使用
    Args:
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        shortcut (Optional[str]): ショートカットの種類 (None または "duplicating")
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: デコーダの入力プロジェクションブロック
    """
    logger.info(f"build_decoder_project_in_block {in_channels=} {out_channels=} {shortcut=} {is_video=}")

    block = ConvLayer(
        in_channels=in_channels,
        out_channels=out_channels,
        kernel_size=3,
        stride=1,
        use_bias=True,
        norm=None,
        act_func=None,
        is_video=is_video,
    )
    if shortcut is None:
        pass
    elif shortcut == "duplicating":
        shortcut_block = ChannelDuplicatingPixelShuffleUpSampleLayer(
            in_channels=in_channels, out_channels=out_channels, factor=1
        )
        block = ResidualBlock(block, shortcut_block)
    else:
        raise ValueError(f"shortcut {shortcut} is not supported for decoder project in")
    return block

In [ ]:
def build_upsample_block(
    block_type: str,
    in_channels: int,
    out_channels: int,
    shortcut: Optional[str],
    is_video: bool,
    temporal_upsample: bool = False,
) -> nn.Module:
    """
    アップサンプリングブロックを構築する
    build_decoder_project_out_block, Decoderで使用

    Args:
        block_type (str): ブロックの種類 ("ConvPixelShuffle" または "InterpolateConv")
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        shortcut (Optional[str]): ショートカットの種類 (None または "duplicating")
        is_video (bool): 3D畳み込みを使用するかどうか
        temporal_upsample (bool, optional):
            時間方向のアップサンプリングを行うかどうか
    Returns:
        nn.Module: アップサンプリングブロック
    """
    logger.info(f"build_upsample_block {block_type=} {in_channels=} {out_channels=} {shortcut=} {is_video=} {temporal_upsample=}")
    if block_type == "ConvPixelShuffle":
        if is_video:
            raise NotImplementedError("ConvPixelShuffle upsample is not supported for video")
        block = ConvPixelShuffleUpSampleLayer(
            in_channels=in_channels, out_channels=out_channels, kernel_size=3, factor=2
        )
    elif block_type == "InterpolateConv":
        block = InterpolateConvUpSampleLayer(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            factor=2,
            is_video=is_video,
            temporal_upsample=temporal_upsample,
        )
    else:
        raise ValueError(f"block_type {block_type} is not supported for upsampling")
    if shortcut is None:
        pass
    elif shortcut == "duplicating":
        shortcut_block = ChannelDuplicatingPixelShuffleUpSampleLayer(
            in_channels=in_channels, out_channels=out_channels, factor=2, temporal_upsample=temporal_upsample
        )
        block = ResidualBlock(block, shortcut_block)
    else:
        raise ValueError(f"shortcut {shortcut} is not supported for upsample")
    return block

In [ ]:
def build_decoder_project_out_block(
    in_channels: int,
    out_channels: int,
    factor: int,
    upsample_block_type: str,
    norm: Optional[str],
    act: Optional[str],
    is_video: bool,
):
    """
    デコーダの出力プロジェクションブロックを構築する

    Args:
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        factor (int): アップサンプリングファクター
        upsample_block_type (str): アップサンプリングブロックの種類
        norm (Optional[str]): 正規化層の種類
        act (Optional[str]): 活性化関数の種類
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: デコーダの出力プロジェクションブロック
    """
    logger.info(f"build_decoder_project_out_block {in_channels=} {out_channels=} {factor=} {upsample_block_type=} {norm=} {act=} {is_video=}")

    layers: list[nn.Module] = [
        build_norm(norm, in_channels),
        build_act(act),
    ]
    if factor == 1:
        layers.append(
            ConvLayer(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=1,
                use_bias=True,
                norm=None,
                act_func=None,
                is_video=is_video,
            )
        )
    elif factor == 2:
        if is_video:
            raise NotImplementedError("Upsample during project_out is not supported for video")
        layers.append(
            build_upsample_block(
                block_type=upsample_block_type, in_channels=in_channels, out_channels=out_channels, shortcut=None
            )
        )
    else:
        raise ValueError(f"upsample factor {factor} is not supported for decoder project out")
    return OpSequential(layers)

In [ ]:
@dataclass
class DecoderConfig:
    in_channels: int = MISSING
    latent_channels: int = MISSING
    in_shortcut: Optional[str] = "duplicating"
    width_list: tuple[int, ...] = (128, 256, 512, 512, 1024, 1024)
    depth_list: tuple[int, ...] = (2, 2, 2, 2, 2, 2)
    block_type: Any = "ResBlock"
    norm: Any = "rms2d"
    act: Any = "silu"
    upsample_block_type: str = "ConvPixelShuffle"
    upsample_match_channel: bool = True
    upsample_shortcut: str = "duplicating"
    out_norm: str = "rms2d"
    out_act: str = "relu"
    is_video: bool = False
    temporal_upsample: tuple[bool, ...] = ()

In [ ]:
class Decoder(nn.Module):
    """
    デコーダ
    DCAEで使用
    """

    def __init__(self, cfg: DecoderConfig):
        """
        Args:
            cfg (DecoderConfig): デコーダの設定
        """
        logger.info(f"Decoder.__init__ {cfg=}")
        super().__init__()
        self.cfg = cfg
        num_stages = len(cfg.width_list)
        self.num_stages = num_stages
        assert len(cfg.depth_list) == num_stages
        assert len(cfg.width_list) == num_stages
        assert isinstance(cfg.block_type, str) or (
            isinstance(cfg.block_type, list) and len(cfg.block_type) == num_stages
        )
        assert isinstance(cfg.norm, str) or (isinstance(cfg.norm, list) and len(cfg.norm) == num_stages)
        assert isinstance(cfg.act, str) or (isinstance(cfg.act, list) and len(cfg.act) == num_stages)

        self.project_in = build_decoder_project_in_block(
            in_channels=cfg.latent_channels,
            out_channels=cfg.width_list[-1],
            shortcut=cfg.in_shortcut,
            is_video=cfg.is_video,
        )

        self.stages: list[OpSequential] = []
        for stage_id, (width, depth) in reversed(list(enumerate(zip(cfg.width_list, cfg.depth_list)))):
            stage = []
            if stage_id < num_stages - 1 and depth > 0:
                upsample_block = build_upsample_block(
                    block_type=cfg.upsample_block_type,
                    in_channels=cfg.width_list[stage_id + 1],
                    out_channels=width if cfg.upsample_match_channel else cfg.width_list[stage_id + 1],
                    shortcut=cfg.upsample_shortcut,
                    is_video=cfg.is_video,
                    temporal_upsample=cfg.temporal_upsample[stage_id] if cfg.temporal_upsample != [] else False,
                )
                stage.append(upsample_block)

            block_type = cfg.block_type[stage_id] if isinstance(cfg.block_type, list) else cfg.block_type
            norm = cfg.norm[stage_id] if isinstance(cfg.norm, list) else cfg.norm
            act = cfg.act[stage_id] if isinstance(cfg.act, list) else cfg.act
            stage.extend(
                build_stage_main(
                    width=width,
                    depth=depth,
                    block_type=block_type,
                    norm=norm,
                    act=act,
                    input_width=(
                        width if cfg.upsample_match_channel else cfg.width_list[min(stage_id + 1, num_stages - 1)]
                    ),
                    is_video=cfg.is_video,
                )
            )
            self.stages.insert(0, OpSequential(stage))
        self.stages = nn.ModuleList(self.stages)

        self.project_out = build_decoder_project_out_block(
            in_channels=cfg.width_list[0] if cfg.depth_list[0] > 0 else cfg.width_list[1],
            out_channels=cfg.in_channels,
            factor=1 if cfg.depth_list[0] > 0 else 2,
            upsample_block_type=cfg.upsample_block_type,
            norm=cfg.out_norm,
            act=cfg.out_act,
            is_video=cfg.is_video,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"Decoder.forward {x.shape=}")
        x = auto_grad_checkpoint(self.project_in, x)
        for stage in reversed(self.stages):
            if len(stage.op_list) == 0:
                continue
            # x = stage(x)
            x = auto_grad_checkpoint(stage, x)

        if self.disc_off_grad_ckpt:
            x = self.project_out(x)
        else:
            x = auto_grad_checkpoint(self.project_out, x)
        return x

### DC-AE

In [ ]:
@dataclass
class DCAEConfig:
    in_channels: int = 3
    latent_channels: int = 32
    time_compression_ratio: int = 1
    spatial_compression_ratio: int = 32
    encoder: EncoderConfig = field(
        default_factory=lambda: EncoderConfig(in_channels="${..in_channels}", latent_channels="${..latent_channels}")
    )
    decoder: DecoderConfig = field(
        default_factory=lambda: DecoderConfig(in_channels="${..in_channels}", latent_channels="${..latent_channels}")
    )
    use_quant_conv: bool = False

    pretrained_path: Optional[str] = None
    pretrained_source: str = "dc-ae"

    scaling_factor: Optional[float] = None
    is_image_model: bool = False

    is_training: bool = False  # NOTE: set to True in vae train config

    use_spatial_tiling: bool = False
    use_temporal_tiling: bool = False
    spatial_tile_size: int = 256
    temporal_tile_size: int = 32
    tile_overlap_factor: float = 0.25

In [ ]:
def init_modules(model: Union[nn.Module, list[nn.Module]], init_type="trunc_normal") -> None:
    """
    モデルの重みを初期化する

    Args:
        model (nn.Module or list[nn.Module]): 初期化するモデルまたはモデルのリスト
        init_type (str, optional): 初期化の種類 ("trunc_normal@std" または "normal@std")
    """
    logger.info(f"init_modules {init_type=}")

    _DEFAULT_INIT_PARAM = {"trunc_normal": 0.02}

    if isinstance(model, list):
        for sub_module in model:
            init_modules(sub_module, init_type)
    else:
        init_params = init_type.split("@")
        init_params = float(init_params[1]) if len(init_params) > 1 else None

        if init_type.startswith("trunc_normal"):
            init_func = lambda param: nn.init.trunc_normal_(
                param, std=(_DEFAULT_INIT_PARAM["trunc_normal"] if init_params is None else init_params)
            )
        elif init_type.startswith("normal"):
            init_func = lambda param: nn.init.normal_(
                param, std=(_DEFAULT_INIT_PARAM["trunc_normal"] if init_params is None else init_params)
            )
        else:
            raise NotImplementedError

        for m in model.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear, nn.ConvTranspose2d)):
                init_func(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.Embedding):
                init_func(m.weight)
            elif isinstance(m, (_BatchNorm, nn.GroupNorm, nn.LayerNorm)):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            else:
                weight = getattr(m, "weight", None)
                bias = getattr(m, "bias", None)
                if isinstance(weight, torch.nn.Parameter):
                    init_func(weight)
                if isinstance(bias, torch.nn.Parameter):
                    bias.data.zero_()

In [ ]:
class DCAE(nn.Module):
    """
    Deep Compressive Autoencoder (DCAE)
    """

    def __init__(self, cfg: DCAEConfig):
        """
        Args:
            cfg (DCAEConfig): DCAEの設定
        """
        logger.info(f"DCAE.__init__ {cfg=}")
        super().__init__()
        self.cfg = cfg
        self.encoder = Encoder(cfg.encoder)
        self.decoder = Decoder(cfg.decoder)
        self.scaling_factor = cfg.scaling_factor
        self.time_compression_ratio = cfg.time_compression_ratio
        self.spatial_compression_ratio = cfg.spatial_compression_ratio
        self.use_spatial_tiling = cfg.use_spatial_tiling
        self.use_temporal_tiling = cfg.use_temporal_tiling
        self.spatial_tile_size = cfg.spatial_tile_size
        self.temporal_tile_size = cfg.temporal_tile_size
        assert (
            cfg.spatial_tile_size // cfg.spatial_compression_ratio
        ), f"spatial tile size {cfg.spatial_tile_size} must be divisible by spatial compression of {cfg.spatial_compression_ratio}"
        self.spatial_tile_latent_size = cfg.spatial_tile_size // cfg.spatial_compression_ratio
        assert (
            cfg.temporal_tile_size // cfg.time_compression_ratio
        ), f"temporal tile size {cfg.temporal_tile_size} must be divisible by temporal compression of {cfg.time_compression_ratio}"
        self.temporal_tile_latent_size = cfg.temporal_tile_size // cfg.time_compression_ratio
        self.tile_overlap_factor = cfg.tile_overlap_factor
        if self.cfg.pretrained_path is not None:
            self.load_model()

        self.to(torch.float32)
        init_modules(self, init_type="trunc_normal")

    def load_model(self):
        if self.cfg.pretrained_source == "dc-ae":
            state_dict = torch.load(self.cfg.pretrained_path, map_location="cpu", weights_only=True)["state_dict"]
            self.load_state_dict(state_dict)
        else:
            raise NotImplementedError

    def get_last_layer(self):
        return self.decoder.project_out.op_list[2].conv.weight

    # @property
    # def spatial_compression_ratio(self) -> int:
    #     return 2 ** (self.decoder.num_stages - 1)

    def encode_single(self, x: torch.Tensor, is_video_encoder: bool = False) -> torch.Tensor:
        assert x.shape[0] == 1
        is_video = x.dim() == 5
        if is_video and not is_video_encoder:
            b, c, f, h, w = x.shape
            x = x.permute(0, 2, 1, 3, 4).reshape(-1, c, h, w)
        z = self.encoder(x)

        if is_video and not is_video_encoder:
            z = z.unsqueeze(dim=0).permute(0, 2, 1, 3, 4)

        if self.scaling_factor is not None:
            z = z / self.scaling_factor

        return z

    def _encode(self, x: torch.Tensor) -> torch.Tensor:
        if self.cfg.is_training:
            return self.encoder(x)
        is_video_encoder = self.encoder.cfg.is_video if self.encoder.cfg.is_video is not None else False
        x_ret = []
        for i in range(x.shape[0]):
            x_ret.append(self.encode_single(x[i : i + 1], is_video_encoder))
        return torch.cat(x_ret, dim=0)

    def blend_v(self, a: torch.Tensor, b: torch.Tensor, blend_extent: int) -> torch.Tensor:
        blend_extent = min(a.shape[-2], b.shape[-2], blend_extent)
        for y in range(blend_extent):
            b[:, :, :, y, :] = a[:, :, :, -blend_extent + y, :] * (1 - y / blend_extent) + b[:, :, :, y, :] * (
                y / blend_extent
            )
        return b

    def blend_h(self, a: torch.Tensor, b: torch.Tensor, blend_extent: int) -> torch.Tensor:
        blend_extent = min(a.shape[-1], b.shape[-1], blend_extent)
        for x in range(blend_extent):
            b[:, :, :, :, x] = a[:, :, :, :, -blend_extent + x] * (1 - x / blend_extent) + b[:, :, :, :, x] * (
                x / blend_extent
            )
        return b

    def blend_t(self, a: torch.Tensor, b: torch.Tensor, blend_extent: int) -> torch.Tensor:
        blend_extent = min(a.shape[-3], b.shape[-3], blend_extent)
        for x in range(blend_extent):
            b[:, :, x, :, :] = a[:, :, -blend_extent + x, :, :] * (1 - x / blend_extent) + b[:, :, x, :, :] * (
                x / blend_extent
            )
        return b

    def spatial_tiled_encode(self, x: torch.Tensor) -> torch.Tensor:
        net_size = int(self.spatial_tile_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.spatial_tile_latent_size * self.tile_overlap_factor)
        row_limit = self.spatial_tile_latent_size - blend_extent

        # Split video into tiles and encode them separately.
        rows = []
        for i in range(0, x.shape[-2], net_size):
            row = []
            for j in range(0, x.shape[-1], net_size):
                tile = x[:, :, :, i : i + self.spatial_tile_size, j : j + self.spatial_tile_size]
                tile = self._encode(tile)
                row.append(tile)
            rows.append(row)
        result_rows = []
        for i, row in enumerate(rows):
            result_row = []
            for j, tile in enumerate(row):
                # blend the above tile and the left tile
                # to the current tile and add the current tile to the result row
                if i > 0:
                    tile = self.blend_v(rows[i - 1][j], tile, blend_extent)
                if j > 0:
                    tile = self.blend_h(row[j - 1], tile, blend_extent)
                result_row.append(tile[:, :, :, :row_limit, :row_limit])
            result_rows.append(torch.cat(result_row, dim=-1))

        return torch.cat(result_rows, dim=-2)

    def temporal_tiled_encode(self, x: torch.Tensor) -> torch.Tensor:
        overlap_size = int(self.temporal_tile_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.temporal_tile_latent_size * self.tile_overlap_factor)
        t_limit = self.temporal_tile_latent_size - blend_extent

        # Split the video into tiles and encode them separately.
        row = []
        for i in range(0, x.shape[2], overlap_size):
            tile = x[:, :, i : i + self.temporal_tile_size, :, :]
            if self.use_spatial_tiling and (
                tile.shape[-1] > self.spatial_tile_size or tile.shape[-2] > self.spatial_tile_size
            ):
                tile = self.spatial_tiled_encode(tile)
            else:
                tile = self._encode(tile)
            row.append(tile)
        result_row = []
        for i, tile in enumerate(row):
            if i > 0:
                tile = self.blend_t(row[i - 1], tile, blend_extent)
            result_row.append(tile[:, :, :t_limit, :, :])

        return torch.cat(result_row, dim=2)

    def encode(self, x: torch.Tensor) -> torch.Tensor:
        if self.use_temporal_tiling and x.shape[2] > self.temporal_tile_size:
            return self.temporal_tiled_encode(x)
        elif self.use_spatial_tiling and (x.shape[-1] > self.spatial_tile_size or x.shape[-2] > self.spatial_tile_size):
            return self.spatial_tiled_encode(x)
        else:
            return self._encode(x)

    def spatial_tiled_decode(self, z: torch.FloatTensor) -> torch.Tensor:
        net_size = int(self.spatial_tile_latent_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.spatial_tile_size * self.tile_overlap_factor)
        row_limit = self.spatial_tile_size - blend_extent

        # Split z into overlapping tiles and decode them separately.
        # The tiles have an overlap to avoid seams between tiles.
        rows = []
        for i in range(0, z.shape[-2], net_size):
            row = []
            for j in range(0, z.shape[-1], net_size):
                tile = z[:, :, :, i : i + self.spatial_tile_latent_size, j : j + self.spatial_tile_latent_size]
                decoded = self._decode(tile)
                row.append(decoded)
            rows.append(row)
        result_rows = []
        for i, row in enumerate(rows):
            result_row = []
            for j, tile in enumerate(row):
                # blend the above tile and the left tile
                # to the current tile and add the current tile to the result row
                if i > 0:
                    tile = self.blend_v(rows[i - 1][j], tile, blend_extent)
                if j > 0:
                    tile = self.blend_h(row[j - 1], tile, blend_extent)
                result_row.append(tile[:, :, :, :row_limit, :row_limit])
            result_rows.append(torch.cat(result_row, dim=-1))

        return torch.cat(result_rows, dim=-2)

    def temporal_tiled_decode(self, z: torch.Tensor) -> torch.Tensor:
        overlap_size = int(self.temporal_tile_latent_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.temporal_tile_size * self.tile_overlap_factor)
        t_limit = self.temporal_tile_size - blend_extent

        row = []
        for i in range(0, z.shape[2], overlap_size):
            tile = z[:, :, i : i + self.temporal_tile_latent_size, :, :]
            if self.use_spatial_tiling and (
                tile.shape[-1] > self.spatial_tile_latent_size or tile.shape[-2] > self.spatial_tile_latent_size
            ):
                decoded = self.spatial_tiled_decode(tile)
            else:
                decoded = self._decode(tile)
            row.append(decoded)
        result_row = []
        for i, tile in enumerate(row):
            if i > 0:
                tile = self.blend_t(row[i - 1], tile, blend_extent)
            result_row.append(tile[:, :, :t_limit, :, :])

        return torch.cat(result_row, dim=2)

    def decode_single(self, z: torch.Tensor, is_video_decoder: bool = False) -> torch.Tensor:
        assert z.shape[0] == 1
        is_video = z.dim() == 5
        if is_video and not is_video_decoder:
            b, c, f, h, w = z.shape
            z = z.permute(0, 2, 1, 3, 4).reshape(-1, c, h, w)
        if self.scaling_factor is not None:
            z = z * self.scaling_factor

        x = self.decoder(z)

        if is_video and not is_video_decoder:
            x = x.unsqueeze(dim=0).permute(0, 2, 1, 3, 4)
        return x

    def _decode(self, z: torch.Tensor) -> torch.Tensor:
        if self.cfg.is_training:
            return self.decoder(z)
        is_video_decoder = self.decoder.cfg.is_video if self.decoder.cfg.is_video is not None else False
        x_ret = []
        for i in range(z.shape[0]):
            x_ret.append(self.decode_single(z[i : i + 1], is_video_decoder))
        return torch.cat(x_ret, dim=0)

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        if self.use_temporal_tiling and z.shape[2] > self.temporal_tile_latent_size:
            return self.temporal_tiled_decode(z)
        elif self.use_spatial_tiling and (
            z.shape[-1] > self.spatial_tile_latent_size or z.shape[-2] > self.spatial_tile_latent_size
        ):
            return self.spatial_tiled_decode(z)
        else:
            return self._decode(z)

    def forward(self, x: torch.Tensor) -> tuple[Any, Tensor, dict[Any, Any]]:
        x_type = x.dtype
        is_image_model = self.cfg.__dict__.get("is_image_model", False)
        x = x.to(self.encoder.project_in.conv.weight.dtype)

        if is_image_model:
            b, c, _, h, w = x.shape
            x = x.permute(0, 2, 1, 3, 4).reshape(-1, c, h, w)

        z = self.encode(x)
        dec = self.decode(z)

        if is_image_model:
            dec = dec.reshape(b, 1, c, h, w).permute(0, 2, 1, 3, 4)
            z = z.unsqueeze(dim=0).permute(0, 2, 1, 3, 4)

        dec = dec.to(x_type)
        return dec, None, z

    def get_latent_size(self, input_size: list[int]) -> list[int]:
        latent_size = []
        # T
        latent_size.append((input_size[0] - 1) // self.time_compression_ratio + 1)
        # H, w
        for i in range(1, 3):
            latent_size.append((input_size[i] - 1) // self.spatial_compression_ratio + 1)
        return latent_size

### モジュール化

In [ ]:
def dc_ae_f32(name: str, pretrained_path: str) -> DCAEConfig:
    if name in ["dc-ae-f32t4c128"]:
        cfg_str = (
            "time_compression_ratio=4 "
            "spatial_compression_ratio=32 "
            "encoder.block_type=[ResBlock,ResBlock,ResBlock,EViTS5_GLU,EViTS5_GLU,EViTS5_GLU] "
            "encoder.width_list=[128,256,512,512,1024,1024] encoder.depth_list=[2,2,2,3,3,3] "
            "encoder.downsample_block_type=Conv "
            "encoder.norm=rms3d "
            "encoder.is_video=True "
            "decoder.block_type=[ResBlock,ResBlock,ResBlock,EViTS5_GLU,EViTS5_GLU,EViTS5_GLU] "
            "decoder.width_list=[128,256,512,512,1024,1024] decoder.depth_list=[3,3,3,3,3,3] "
            "decoder.upsample_block_type=InterpolateConv "
            "decoder.norm=rms3d decoder.act=silu decoder.out_norm=rms3d "
            "decoder.is_video=True "
            "encoder.temporal_downsample=[False,False,False,True,True,False] "
            "decoder.temporal_upsample=[False,False,False,True,True,False] "
            "latent_channels=128"
        )  # make sure there is no trailing blankspace in the last line
    else:
        raise NotImplementedError
    cfg = OmegaConf.from_dotlist(cfg_str.split(" "))
    cfg: DCAEConfig = OmegaConf.to_object(OmegaConf.merge(OmegaConf.structured(DCAEConfig), cfg))
    cfg.pretrained_path = pretrained_path
    return cfg

In [ ]:
REGISTERED_DCAE_MODEL: dict[str, tuple[Callable, Optional[str]]] = {
    "dc-ae-f32t4c128": (dc_ae_f32, None),
}

In [ ]:
def create_dc_ae_model_cfg(name: str, pretrained_path: Optional[str] = None) -> DCAEConfig:
    """
    登録されたDCAEモデル設定を作成する

    Args:
        name (str): 登録されたDCAEモデルの名前
        pretrained_path (Optional[str], optional): 事前学習済みモデルのパス
    Returns:
        DCAEConfig: DCAEモデルの設定
    """
    assert name in REGISTERED_DCAE_MODEL, f"{name} is not supported"
    dc_ae_cls, default_pt_path = REGISTERED_DCAE_MODEL[name]
    pretrained_path = default_pt_path if pretrained_path is None else pretrained_path
    model_cfg = dc_ae_cls(name, pretrained_path)
    return model_cfg

In [ ]:
class DCAE_HF(DCAE, PyTorchModelHubMixin):
    """
    HuggingFace Hub対応DCAEモデル
    """
    def __init__(self, model_name: str):
        logger.info(f"DCAE_HF.__init__ {model_name=}")
        cfg = create_dc_ae_model_cfg(model_name)
        DCAE.__init__(self, cfg)

In [ ]:
# @MODELS.register_module("dc_ae")
def DC_AE(
    model_name: str,
    device_map: str | torch.device = "cuda",
    torch_dtype: torch.dtype = torch.bfloat16,
    from_scratch: bool = False,
    from_pretrained: str | None = None,
    is_training: bool = False,
    use_spatial_tiling: bool = False,
    use_temporal_tiling: bool = False,
    spatial_tile_size: int = 256,
    temporal_tile_size: int = 32,
    tile_overlap_factor: float = 0.25,
    scaling_factor: float = None,
    disc_off_grad_ckpt: bool = False,
) -> DCAE_HF:
    """
    Deep Compressive Autoencoder (DCAE)モデルを作成する

    Args:
        model_name (str): モデルの名前
        device_map (str or torch.device, optional): モデルを配置するデバイス
        torch_dtype (torch.dtype, optional): モデルのデータ型
        from_scratch (bool, optional): ランダム初期化からモデルを作成するかどうか
        from_pretrained (str or None, optional): 事前学習済みモデルのパス
        is_training (bool, optional): モデルをトレーニングモードに設定するかどうか
        use_spatial_tiling (bool, optional): 空間タイル処理を使用するかどうか
        use_temporal_tiling (bool, optional): 時間タイル処理を使用するかどうか
        spatial_tile_size (int, optional): 空間タイルのサイズ
        temporal_tile_size (int, optional): 時間タイルのサイズ
        tile_overlap_factor (float, optional): タイルの重なり係数
        scaling_factor (float, optional): スケーリングファクター
        disc_off_grad_ckpt (bool, optional): 勾配チェックポイントを無効にするかどうか
    Returns:
        DCAE_HF: DCAEモデル
    """
    logger.info(f"DC_AE {model_name=} {device_map=} {torch_dtype=} {from_scratch=} {from_pretrained=} {is_training=} {use_spatial_tiling=} {use_temporal_tiling=} {spatial_tile_size=} {temporal_tile_size=} {tile_overlap_factor=} {scaling_factor=} {disc_off_grad_ckpt=}")

    if not from_scratch:
        model = DCAE_HF.from_pretrained(model_name).to(device_map, torch_dtype)
    else:
        model = DCAE_HF(model_name).to(device_map, torch_dtype)

    if from_pretrained is not None:
        model = load_checkpoint(model, from_pretrained, device_map=device_map)
        print(f"loaded dc_ae from ckpt path: {from_pretrained}")

    model.cfg.is_training = is_training
    model.use_spatial_tiling = use_spatial_tiling
    model.use_temporal_tiling = use_temporal_tiling
    model.spatial_tile_size = spatial_tile_size
    model.temporal_tile_size = temporal_tile_size
    model.tile_overlap_factor = tile_overlap_factor
    if scaling_factor is not None:
        model.scaling_factor = scaling_factor
    model.decoder.disc_off_grad_ckpt = disc_off_grad_ckpt
    return model

# {'type': 'dc_ae',
#  'model_name': 'dc-ae-f32t4c128',
#  'from_scratch': True,
#  'from_pretrained': None}

DC_AE(
    model_name="dc-ae-f32t4c128",
    from_scratch=True,
    from_pretrained=None,
)

In [ ]:
# (batch_size, channels, frames, height, width)
sample_input = torch.randn(1, 3, 1, 256, 256).to("cuda")

model = DC_AE(
    model_name="dc-ae-f32t4c128",
    from_scratch=True,
    from_pretrained=None,
    is_training=False,
    # use_spatial_tiling=True,
    # use_temporal_tiling=True,
    # spatial_tile_size=256,
    # temporal_tile_size=16,
    # tile_overlap_factor=0.25,
)
output = model(sample_input)
print(output[0].shape)  # Decoded output

## MMDiT

### LigerEmbedND

In [ ]:
def liger_rope(pos: Tensor, dim: int, theta: int) -> Tuple:
    """
    Liger RoPE
    LigerEmbedNDで使用

    Args:
        pos (Tensor): 位置エンコーディングの位置テンソル (..., n)
        dim (int): 埋め込み次元数
        theta (int): RoPEのスケーリングパラメータ
    Returns:
        Tuple: コサインとサインのテンソル (..., n, dim//2)
    """
    logger.info(f"liger_rope {pos.shape=} {dim=} {theta=}")

    assert dim % 2 == 0
    scale = torch.arange(0, dim, 2, dtype=torch.float32, device=pos.device) / dim
    omega = 1.0 / (theta**scale)
    out = torch.einsum("...n,d->...nd", pos, omega)  # (b, seq, dim//2)
    cos = out.cos()
    sin = out.sin()

    return (cos, sin)

In [ ]:
class LigerEmbedND(nn.Module):
    """
    Liger Multi-dimensional RoPE Embedding
    MMDiTModelで使用
    """

    def __init__(self, dim: int, theta: int, axes_dim: list[int]):
        """
        Args:
            dim (int): 埋め込み次元数
            theta (int): RoPEのスケーリングパラメータ
            axes_dim (list[int]): 各軸の次元数
        """
        logger.info(f"LigerEmbedND.__init__ {dim=} {theta=} {axes_dim=}")
        super().__init__()
        self.dim = dim
        self.theta = theta
        self.axes_dim = axes_dim

    def forward(self, ids: Tensor) -> Tensor:
        logger.info(f"LigerEmbedND.forward {ids.shape=}")
        n_axes = ids.shape[-1]
        cos_list = []
        sin_list = []
        for i in range(n_axes):
            cos, sin = liger_rope(ids[..., i], self.axes_dim[i], self.theta)
            cos_list.append(cos)
            sin_list.append(sin)
        cos_emb = torch.cat(cos_list, dim=-1).repeat(1, 1, 2).contiguous()
        sin_emb = torch.cat(sin_list, dim=-1).repeat(1, 1, 2).contiguous()

        return (cos_emb, sin_emb)

### MLPEmbedder

In [ ]:
class MLPEmbedder(nn.Module):
    """
    MLP Embedder
    MMDiTModelで使用
    """
    def __init__(self, in_dim: int, hidden_dim: int):
        logger.info(f"MLPEmbedder.__init__ {in_dim=} {hidden_dim=}")
        super().__init__()
        self.in_layer = nn.Linear(in_dim, hidden_dim, bias=True)
        self.silu = nn.SiLU()
        self.out_layer = nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x: Tensor) -> Tensor:
        logger.info(f"MLPEmbedder.forward {x.shape=}")
        return self.out_layer(self.silu(self.in_layer(x)))

### QKNorm

In [ ]:
class RMSNorm(torch.nn.Module):
    """
    RMS Normalization
    FusedRMSNormで継承
    """

    def __init__(self, dim: int):
        """
        Args:
            dim (int): 埋め込み次元数
        """
        logger.info(f"RMSNorm.__init__ {dim=}")
        super().__init__()
        self.scale = nn.Parameter(torch.ones(dim))

    def forward(self, x: Tensor):
        logger.info(f"RMSNorm.forward {x.shape=}")
        x_dtype = x.dtype
        x = x.float()
        rrms = torch.rsqrt(torch.mean(x**2, dim=-1, keepdim=True) + 1e-6)
        return (x * rrms).to(dtype=x_dtype) * self.scale

In [ ]:
class FusedRMSNorm(RMSNorm):
    """
    Fused RMS Normalization
    QKNormで使用
    """

    def forward(self, x: Tensor):
        logger.info(f"FusedRMSNorm.forward {x.shape=}")
        return LigerRMSNormFunction.apply(
            x,
            self.scale,
            1e-6,
            0.0,
            "llama",
            False,
        )

In [ ]:
class QKNorm(torch.nn.Module):
    """
    Query-Key Normalization
    Self-Attentionで使用
    """

    def __init__(self, dim: int):
        """
        Args:
            dim (int): 埋め込み次元数
        """
        logger.info(f"QKNorm.__init__ {dim=}")
        super().__init__()
        self.query_norm = FusedRMSNorm(dim)
        self.key_norm = FusedRMSNorm(dim)

    def forward(self, q: Tensor, k: Tensor, v: Tensor) -> tuple[Tensor, Tensor]:
        """
        Args:
            q (Tensor): クエリテンソル (..., dim)
            k (Tensor): キーテンソル (..., dim)
            v (Tensor): バリューテンソル (..., dim)
        Returns:
            tuple[Tensor, Tensor]: 正規化されたクエリとキーのテンソル
        """
        logger.info(f"QKNorm.forward {q.shape=} {k.shape=} {v.shape=}")
        q = self.query_norm(q)
        k = self.key_norm(k)
        return q.to(v), k.to(v)

### アテンション機構

In [ ]:
def flash_attn_func(q: Tensor, k: Tensor, v: Tensor) -> Tensor:
    """
    Flash Attentionを使用したアテンション計算
    Args:
        q (Tensor): クエリテンソル (B, H, L, D)
        k (Tensor): キーテンソル (B, H, L, D)
        v (Tensor): バリューテンソル (B, H, L, D)
    Returns:
        Tensor: アテンション出力テンソル (B, L, H, D)
    """
    logger.info(f"flash_attn_func {q.shape=} {k.shape=} {v.shape=}")

    if SUPPORT_FA3:
        return flash_attn_func_v3(q, k, v)[0]
    return flash_attn_func_v2(q, k, v)

In [ ]:
def attention(q: Tensor, k: Tensor, v: Tensor, pe) -> Tensor:
    """
    アテンション計算
    SingleStreamBlockProcessor, DoubleSreamBlockProcessor,
    SelfAttentionで使用
    Args:
        q (Tensor): クエリテンソル (B, L, H, D)
        k (Tensor): キーテンソル (B, L, H, D)
        v (Tensor): バリューテンソル (B, L, H, D)
        pe: 位置エンコーディングテンソルまたはコサイン・サインテンソルのタプル
    Returns:
        Tensor: アテンション出力テンソル (B, L, H*D)
    """
    if isinstance(pe, torch.Tensor):
        q, k = apply_rope(q, k, pe)
    else:
        cos, sin = pe
        q, k = LigerRopeFunction.apply(q, k, cos, sin)
        # to compare with the original implementation
        # k = reverse_rearrange_tensor(k)
    q = rearrange(q, "B H L D -> B L H D")
    k = rearrange(k, "B H L D -> B L H D")
    v = rearrange(v, "B H L D -> B L H D")
    x = flash_attn_func(q, k, v)
    x = rearrange(x, "B L H D -> B L (H D)")

    return x

In [ ]:
class SelfAttention(nn.Module):
    """
    セルフアテンション
    DoubleStreamBlockで使用
    """

    def __init__(self, dim: int, num_heads: int = 8, qkv_bias: bool = False, fused_qkv: bool = True):
        """
        Args:
            dim (int): 埋め込み次元数
            num_heads (int, optional): アテンションヘッド数. Defaults to 8.
            qkv_bias (bool, optional): QKVのバイアスを使用するかどうか. Defaults to False.
            fused_qkv (bool, optional): QKVを融合した線形層を使用するかどうか. Defaults to True.
        """
        logger.info(f"SelfAttention.__init__ {dim=} {num_heads=} {qkv_bias=} {fused_qkv=}")

        super().__init__()
        self.num_heads = num_heads
        self.fused_qkv = fused_qkv
        head_dim = dim // num_heads

        if fused_qkv:
            self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        else:
            self.q_proj = nn.Linear(dim, dim, bias=qkv_bias)
            self.k_proj = nn.Linear(dim, dim, bias=qkv_bias)
            self.v_proj = nn.Linear(dim, dim, bias=qkv_bias)
        self.norm = QKNorm(head_dim)
        self.proj = nn.Linear(dim, dim)

    def forward(self, x: Tensor, pe: Tensor) -> Tensor:
        logger.info(f"SelfAttention.forward {x.shape=} {pe.shape=}")
        if self.fused_qkv:
            qkv = self.qkv(x)
            q, k, v = rearrange(qkv, "B L (K H D) -> K B H L D", K=3, H=self.num_heads)
        else:
            q = rearrange(self.q_proj(x), "B L (H D) -> B L H D", H=self.num_heads)
            k = rearrange(self.k_proj(x), "B L (H D) -> B L H D", H=self.num_heads)
            v = rearrange(self.v_proj(x), "B L (H D) -> B L H D", H=self.num_heads)
        q, k = self.norm(q, k, v)
        if not self.fused_qkv:
            q = rearrange(q, "B L H D -> B H L D")
            k = rearrange(k, "B L H D -> B H L D")
            v = rearrange(v, "B L H D -> B H L D")
        x = attention(q, k, v, pe=pe)
        x = self.proj(x)
        return x

### Modulation

In [ ]:
@dataclass
class ModulationOut:
    shift: Tensor
    scale: Tensor
    gate: Tensor

In [ ]:
class Modulation(nn.Module):
    """
    モジュレーション層
    DoubleStreamBlockで使用
    """

    def __init__(self, dim: int, double: bool):
        logger.info(f"Modulation.__init__ {dim=} {double=}")
        super().__init__()
        self.is_double = double
        self.multiplier = 6 if double else 3
        self.lin = nn.Linear(dim, self.multiplier * dim, bias=True)

    def forward(self, vec: Tensor) -> tuple[ModulationOut, ModulationOut | None]:
        logger.info(f"Modulation.forward {vec.shape=}")
        out = self.lin(nn.functional.silu(vec))[:, None, :].chunk(self.multiplier, dim=-1)

        return (
            ModulationOut(*out[:3]),
            ModulationOut(*out[3:]) if self.is_double else None,
        )

### DoubleStreamBlock

In [ ]:
class DoubleStreamBlockProcessor:
    """
    ダブルストリームブロックプロセッサ
    DoubleStreamBlockで使用
    画像とテキストの両方のストリームを処理する
    """

    def __call__(self, attn: nn.Module, img: Tensor, txt: Tensor, vec: Tensor, pe: Tensor) -> tuple[Tensor, Tensor]:
        logger.info(f"DoubleStreamBlockProcessor.__call__ {img.shape=} {txt.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")

        # attn is the DoubleStreamBlock;
        # process img and txt separately while both is influenced by text vec

        # vec will interact with image latent and text context
        img_mod1, img_mod2 = attn.img_mod(vec)  # get shift, scale, gate for each mod
        txt_mod1, txt_mod2 = attn.txt_mod(vec)

        # prepare image for attention
        img_modulated = attn.img_norm1(img)
        img_modulated = (1 + img_mod1.scale) * img_modulated + img_mod1.shift

        if attn.img_attn.fused_qkv:
            img_qkv = attn.img_attn.qkv(img_modulated)
            img_q, img_k, img_v = rearrange(img_qkv, "B L (K H D) -> K B H L D", K=3, H=attn.num_heads, D=attn.head_dim)
        else:
            img_q = rearrange(attn.img_attn.q_proj(img_modulated), "B L (H D) -> B L H D", H=attn.num_heads)
            img_k = rearrange(attn.img_attn.k_proj(img_modulated), "B L (H D) -> B L H D", H=attn.num_heads)
            img_v = rearrange(attn.img_attn.v_proj(img_modulated), "B L (H D) -> B L H D", H=attn.num_heads)

        img_q, img_k = attn.img_attn.norm(img_q, img_k, img_v)  # RMSNorm for QK Norm as in SD3 paper
        if not attn.img_attn.fused_qkv:
            img_q = rearrange(img_q, "B L H D -> B H L D")
            img_k = rearrange(img_k, "B L H D -> B H L D")
            img_v = rearrange(img_v, "B L H D -> B H L D")

        # prepare txt for attention
        txt_modulated = attn.txt_norm1(txt)
        txt_modulated = (1 + txt_mod1.scale) * txt_modulated + txt_mod1.shift
        if attn.txt_attn.fused_qkv:
            txt_qkv = attn.txt_attn.qkv(txt_modulated)
            txt_q, txt_k, txt_v = rearrange(txt_qkv, "B L (K H D) -> K B H L D", K=3, H=attn.num_heads, D=attn.head_dim)
        else:
            txt_q = rearrange(attn.txt_attn.q_proj(txt_modulated), "B L (H D) -> B L H D", H=attn.num_heads)
            txt_k = rearrange(attn.txt_attn.k_proj(txt_modulated), "B L (H D) -> B L H D", H=attn.num_heads)
            txt_v = rearrange(attn.txt_attn.v_proj(txt_modulated), "B L (H D) -> B L H D", H=attn.num_heads)
        txt_q, txt_k = attn.txt_attn.norm(txt_q, txt_k, txt_v)
        if not attn.txt_attn.fused_qkv:
            txt_q = rearrange(txt_q, "B L H D -> B H L D")
            txt_k = rearrange(txt_k, "B L H D -> B H L D")
            txt_v = rearrange(txt_v, "B L H D -> B H L D")

        # run actual attention, image and text attention are calculated together by concat different attn heads
        q = torch.cat((txt_q, img_q), dim=2)
        k = torch.cat((txt_k, img_k), dim=2)
        v = torch.cat((txt_v, img_v), dim=2)

        attn1 = attention(q, k, v, pe=pe)
        txt_attn, img_attn = attn1[:, : txt_q.shape[2]], attn1[:, txt_q.shape[2] :]

        # calculate the img bloks
        img = img + img_mod1.gate * attn.img_attn.proj(img_attn)
        img = img + img_mod2.gate * attn.img_mlp((1 + img_mod2.scale) * attn.img_norm2(img) + img_mod2.shift)

        # calculate the txt bloks
        txt = txt + txt_mod1.gate * attn.txt_attn.proj(txt_attn)
        txt = txt + txt_mod2.gate * attn.txt_mlp((1 + txt_mod2.scale) * attn.txt_norm2(txt) + txt_mod2.shift)
        return img, txt

In [ ]:
class DoubleStreamBlock(nn.Module):
    """
    ダブルストリームブロック
    MMDiTModelで使用
    画像とテキストの両方のストリームを処理する
    """

    def __init__(self, hidden_size: int, num_heads: int, mlp_ratio: float, qkv_bias: bool = False, fused_qkv: bool = True):
        """
        Args:
            hidden_size (int): 埋め込み次元数
            num_heads (int): アテンションヘッド数
            mlp_ratio (float): MLPの隠れ層の次元数の比率
            qkv_bias (bool, optional): QKVのバイアスを使用するかどうか
            fused_qkv (bool, optional): QKVを融合した線形層を使用するかどうか
        """
        logger.info(f"DoubleStreamBlock.__init__ {hidden_size=} {num_heads=} {mlp_ratio=} {qkv_bias=} {fused_qkv=}")

        super().__init__()
        mlp_hidden_dim = int(hidden_size * mlp_ratio)
        self.num_heads = num_heads
        self.hidden_size = hidden_size
        self.head_dim = hidden_size // num_heads

        # image stream
        self.img_mod = Modulation(hidden_size, double=True)
        self.img_norm1 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        self.img_attn = SelfAttention(dim=hidden_size, num_heads=num_heads, qkv_bias=qkv_bias, fused_qkv=fused_qkv)

        self.img_norm2 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        self.img_mlp = nn.Sequential(
            nn.Linear(hidden_size, mlp_hidden_dim, bias=True),
            nn.GELU(approximate="tanh"),
            nn.Linear(mlp_hidden_dim, hidden_size, bias=True),
        )

        # text stream
        self.txt_mod = Modulation(hidden_size, double=True)
        self.txt_norm1 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        self.txt_attn = SelfAttention(dim=hidden_size, num_heads=num_heads, qkv_bias=qkv_bias, fused_qkv=fused_qkv)

        self.txt_norm2 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        self.txt_mlp = nn.Sequential(
            nn.Linear(hidden_size, mlp_hidden_dim, bias=True),
            nn.GELU(approximate="tanh"),
            nn.Linear(mlp_hidden_dim, hidden_size, bias=True),
        )

        # processor
        processor = DoubleStreamBlockProcessor()
        self.set_processor(processor)

    def set_processor(self, processor) -> None:
        self.processor = processor

    def get_processor(self):
        return self.processor

    def forward(self, img: Tensor, txt: Tensor, vec: Tensor, pe: Tensor, **kwargs) -> tuple[Tensor, Tensor]:
        logger.info(f"DoubleStreamBlock.forward {img.shape=} {txt.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")
        return self.processor(self, img, txt, vec, pe)

#### SingleStreamBlock

In [ ]:
class SingleStreamBlockProcessor:
    """
    シングルストリームブロックプロセッサ
    SingleStreamBlockで使用
    """

    def __call__(self, attn: nn.Module, x: Tensor, vec: Tensor, pe: Tensor) -> Tensor:
        logger.info(f"SingleStreamBlockProcessor.__call__ {x.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")

        mod, _ = attn.modulation(vec)
        x_mod = (1 + mod.scale) * attn.pre_norm(x) + mod.shift
        if attn.fused_qkv:
            qkv, mlp = torch.split(attn.linear1(x_mod), [3 * attn.hidden_size, attn.mlp_hidden_dim], dim=-1)
            q, k, v = rearrange(qkv, "B L (K H D) -> K B H L D", K=3, H=attn.num_heads)
        else:
            q = rearrange(attn.q_proj(x_mod), "B L (H D) -> B L H D", H=attn.num_heads)
            k = rearrange(attn.k_proj(x_mod), "B L (H D) -> B L H D", H=attn.num_heads)
            v, mlp = torch.split(attn.v_mlp(x_mod), [attn.hidden_size, attn.mlp_hidden_dim], dim=-1)
            v = rearrange(v, "B L (H D) -> B L H D", H=attn.num_heads)

        q, k = attn.norm(q, k, v)
        if not attn.fused_qkv:
            q = rearrange(q, "B L H D -> B H L D")
            k = rearrange(k, "B L H D -> B H L D")
            v = rearrange(v, "B L H D -> B H L D")

        # compute attention
        attn_1 = attention(q, k, v, pe=pe)

        # compute activation in mlp stream, cat again and run second linear layer
        output = attn.linear2(torch.cat((attn_1, attn.mlp_act(mlp)), 2))
        output = x + mod.gate * output
        return output

In [ ]:
class SingleStreamBlock(nn.Module):
    """
    シングルストリームブロック
    MMDiTModelで使用

    並列線形層を持つDiTブロックで、変調インターフェースが適応
    https://arxiv.org/abs/2302.05442
    """

    def __init__(
        self,
        hidden_size: int,
        num_heads: int,
        mlp_ratio: float = 4.0,
        qk_scale: float | None = None,
        fused_qkv: bool = True,
    ):
        """
        Args:
            hidden_size (int): 埋め込み次元数
            num_heads (int): アテンションヘッド数
            mlp_ratio (float, optional): MLPの隠れ層の次元数の比率
            qk_scale (float | None, optional): QKスケーリングファクター
            fused_qkv (bool, optional): QKVを融合した線形層を使用するかどうか
        """
        logger.info(f"SingleStreamBlock.__init__ {hidden_size=} {num_heads=} {mlp_ratio=} {qk_scale=} {fused_qkv=}")
        super().__init__()
        self.hidden_dim = hidden_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        self.scale = qk_scale or self.head_dim**-0.5
        self.fused_qkv = fused_qkv

        self.mlp_hidden_dim = int(hidden_size * mlp_ratio)
        if fused_qkv:
            # qkv and mlp_in
            self.linear1 = nn.Linear(hidden_size, hidden_size * 3 + self.mlp_hidden_dim)
        else:
            self.q_proj = nn.Linear(hidden_size, hidden_size)
            self.k_proj = nn.Linear(hidden_size, hidden_size)
            self.v_mlp = nn.Linear(hidden_size, hidden_size + self.mlp_hidden_dim)

        # proj and mlp_out
        self.linear2 = nn.Linear(hidden_size + self.mlp_hidden_dim, hidden_size)

        self.norm = QKNorm(self.head_dim)

        self.hidden_size = hidden_size
        self.pre_norm = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)

        self.mlp_act = nn.GELU(approximate="tanh")
        self.modulation = Modulation(hidden_size, double=False)

        processor = SingleStreamBlockProcessor()
        self.set_processor(processor)

    def set_processor(self, processor) -> None:
        self.processor = processor

    def get_processor(self):
        return self.processor

    def forward(self, x: Tensor, vec: Tensor, pe: Tensor, **kwargs) -> Tensor:
        logger.info(f"SingleStreamBlock.forward {x.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")
        return self.processor(self, x, vec, pe)

#### MMDiT

In [ ]:
@torch.compile(mode="max-autotune-no-cudagraphs", dynamic=True)
def timestep_embedding(t: Tensor, dim, max_period=10000, time_factor: float = 1000.0):
    """
    サイン波タイムステップ埋め込みを作成する

    Args:
        t (Tensor): バッチ要素ごとに1つのNインデックスを持つ1-Dテンソル。これらは分数値である可能性があります。
        dim (int): 出力の次元。
        max_period (int, optional): 埋め込みの最小周波数を制御します。デフォルトは10000。
        time_factor (float, optional): 時間スケーリングファクター。デフォルトは1000.0。
    Returns:
        Tensor: 埋め込みテンソルの形状は(t.shape[0], dim)
    """
    logger.info(f"timestep_embedding {t.shape=} {dim=} {max_period=} {time_factor=}")

    t = time_factor * t
    half = dim // 2
    freqs = torch.exp(-math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half).to(t.device)

    args = t[:, None].float() * freqs[None]
    embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
    if dim % 2:
        embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
    if torch.is_floating_point(t):
        embedding = embedding.to(t)
    return embedding

In [ ]:
class LastLayer(nn.Module):
    """
    DCAEデコーダの最後の層
    画像パッチを生成するために使用される
    """

    def __init__(self, hidden_size: int, patch_size: int, out_channels: int):
        """
        Args:
            hidden_size (int): 隠れ層の次元数
            patch_size (int): パッチのサイズ
            out_channels (int): 出力チャネル数
        """
        logger.info(f"LastLayer.__init__ {hidden_size=} {patch_size=} {out_channels=}")

        super().__init__()
        self.norm_final = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        self.linear = nn.Linear(hidden_size, patch_size * patch_size * out_channels, bias=True)
        self.adaLN_modulation = nn.Sequential(nn.SiLU(), nn.Linear(hidden_size, 2 * hidden_size, bias=True))

    def forward(self, x: Tensor, vec: Tensor) -> Tensor:
        """
        Args:
            x (Tensor): 入力テンソル (B, N, hidden_size)
            vec (Tensor): モジュレーションベクトル (B, hidden_size)
        Returns:
            Tensor: 出力テンソル (B, N, patch_size * patch_size * out_channels)
        """
        logger.info(f"LastLayer.forward {x.shape=} {vec.shape=}")
        shift, scale = self.adaLN_modulation(vec).chunk(2, dim=1)
        x = (1 + scale[:, None, :]) * self.norm_final(x) + shift[:, None, :]
        x = self.linear(x)
        return x


In [ ]:
@dataclass
class MMDiTConfig:
    model_type = "MMDiT"
    from_pretrained: str
    cache_dir: str
    in_channels: int
    vec_in_dim: int
    context_in_dim: int
    hidden_size: int
    mlp_ratio: float
    num_heads: int
    depth: int
    depth_single_blocks: int
    axes_dim: list[int]
    theta: int
    qkv_bias: bool
    guidance_embed: bool
    cond_embed: bool = False
    fused_qkv: bool = True
    grad_ckpt_settings: tuple[int, int] | None = None
    use_liger_rope: bool = False
    patch_size: int = 2

    def get(self, attribute_name, default=None):
        return getattr(self, attribute_name, default)

    def __contains__(self, attribute_name):
        return hasattr(self, attribute_name)

In [ ]:
class MMDiTModel(nn.Module):
    """
    マルチモーダルディフュージョントランスフォーマーモデル
    画像とテキストの両方のストリームを処理する
    画像生成タスクに使用される
    """
    config_class = MMDiTConfig

    def __init__(self, config: MMDiTConfig):
        """
        Args:
            config (MMDiTConfig): モデルの設定
        """
        logger.info(f"MMDiTModel.__init__ {config=}")

        super().__init__()

        self.config = config
        self.in_channels = config.in_channels
        self.out_channels = self.in_channels
        self.patch_size = config.patch_size

        if config.hidden_size % config.num_heads != 0:
            raise ValueError(
                f"Hidden size {config.hidden_size} must be divisible by num_heads {config.num_heads}"
            )

        pe_dim = config.hidden_size // config.num_heads
        if sum(config.axes_dim) != pe_dim:
            raise ValueError(
                f"Got {config.axes_dim} but expected positional dim {pe_dim}"
            )

        self.hidden_size = config.hidden_size
        self.num_heads = config.num_heads
        pe_embedder_cls = LigerEmbedND if config.use_liger_rope else EmbedND
        self.pe_embedder = pe_embedder_cls(
            dim=pe_dim, theta=config.theta, axes_dim=config.axes_dim
        )

        self.img_in = nn.Linear(self.in_channels, self.hidden_size, bias=True)
        self.time_in = MLPEmbedder(in_dim=256, hidden_dim=self.hidden_size)
        self.vector_in = MLPEmbedder(config.vec_in_dim, self.hidden_size)
        self.guidance_in = (
            MLPEmbedder(in_dim=256, hidden_dim=self.hidden_size)
            if config.guidance_embed
            else nn.Identity()
        )
        self.cond_in = (
            nn.Linear(
                self.in_channels + self.patch_size**2, self.hidden_size, bias=True
            )
            if config.cond_embed
            else nn.Identity()
        )
        self.txt_in = nn.Linear(config.context_in_dim, self.hidden_size)

        self.double_blocks = nn.ModuleList(
            [
                DoubleStreamBlock(
                    self.hidden_size,
                    self.num_heads,
                    mlp_ratio=config.mlp_ratio,
                    qkv_bias=config.qkv_bias,
                    fused_qkv=config.fused_qkv,
                )
                for _ in range(config.depth)
            ]
        )

        self.single_blocks = nn.ModuleList(
            [
                SingleStreamBlock(
                    self.hidden_size,
                    self.num_heads,
                    mlp_ratio=config.mlp_ratio,
                    fused_qkv=config.fused_qkv,
                )
                for _ in range(config.depth_single_blocks)
            ]
        )

        self.final_layer = LastLayer(self.hidden_size, 1, self.out_channels)
        self.initialize_weights()

        if self.config.grad_ckpt_settings:
            self.forward = self.forward_selective_ckpt
        else:
            self.forward = self.forward_ckpt
        self._input_requires_grad = False

    def initialize_weights(self):
        if self.config.cond_embed:
            nn.init.zeros_(self.cond_in.weight)
            nn.init.zeros_(self.cond_in.bias)

    def prepare_block_inputs(
        self,
        img: Tensor,
        img_ids: Tensor,
        txt: Tensor,  # t5 encoded vec
        txt_ids: Tensor,
        timesteps: Tensor,
        y_vec: Tensor,  # clip encoded vec
        cond: Tensor = None,
        guidance: Tensor | None = None,
    ):
        """
        obtain the processed:
            img: projected noisy img latent,
            txt: text context (from t5),
            vec: clip encoded vector,
            pe: the positional embeddings for concatenated img and txt
        """
        if img.ndim != 3 or txt.ndim != 3:
            raise ValueError("Input img and txt tensors must have 3 dimensions.")

        # running on sequences img
        print("img shape before img_in:", img.shape)
        img = self.img_in(img)
        if self.config.cond_embed:
            if cond is None:
                raise ValueError("Didn't get conditional input for conditional model.")
            img = img + self.cond_in(cond)

        vec = self.time_in(timestep_embedding(timesteps, 256))
        if self.config.guidance_embed:
            if guidance is None:
                raise ValueError(
                    "Didn't get guidance strength for guidance distilled model."
                )
            vec = vec + self.guidance_in(timestep_embedding(guidance, 256))
        vec = vec + self.vector_in(y_vec)

        txt = self.txt_in(txt)

        # concat: 4096 + t*h*2/4
        ids = torch.cat((txt_ids, img_ids), dim=1)
        pe = self.pe_embedder(ids)

        if self._input_requires_grad:
            # we only apply lora to double/single blocks, thus we only need to enable grad for these inputs
            img.requires_grad_()
            txt.requires_grad_()

        return img, txt, vec, pe

    def enable_input_require_grads(self):
        """Fit peft lora. This method should not be called manually."""
        self._input_requires_grad = True

    def forward_ckpt(
        self,
        img: Tensor,
        img_ids: Tensor,
        txt: Tensor,
        txt_ids: Tensor,
        timesteps: Tensor,
        y_vec: Tensor,
        cond: Tensor = None,
        guidance: Tensor | None = None,
        **kwargs,
    ) -> Tensor:
        img, txt, vec, pe = self.prepare_block_inputs(
            img, img_ids, txt, txt_ids, timesteps, y_vec, cond, guidance
        )

        for block in self.double_blocks:
            img, txt = auto_grad_checkpoint(block, img, txt, vec, pe)

        img = torch.cat((txt, img), 1)
        for block in self.single_blocks:
            img = auto_grad_checkpoint(block, img, vec, pe)
        img = img[:, txt.shape[1] :, ...]

        img = self.final_layer(img, vec)  # (N, T, patch_size ** 2 * out_channels)
        return img

    def forward_selective_ckpt(
        self,
        img: Tensor,
        img_ids: Tensor,
        txt: Tensor,
        txt_ids: Tensor,
        timesteps: Tensor,
        y_vec: Tensor,
        cond: Tensor = None,
        guidance: Tensor | None = None,
        **kwargs,
    ) -> Tensor:
        img, txt, vec, pe = self.prepare_block_inputs(
            img, img_ids, txt, txt_ids, timesteps, y_vec, cond, guidance
        )

        ckpt_depth_double = self.config.grad_ckpt_settings[0]
        for block in self.double_blocks[:ckpt_depth_double]:
            img, txt = auto_grad_checkpoint(block, img, txt, vec, pe)

        for block in self.double_blocks[ckpt_depth_double:]:
            img, txt = block(img, txt, vec, pe)

        ckpt_depth_single = self.config.grad_ckpt_settings[1]
        img = torch.cat((txt, img), 1)
        for block in self.single_blocks[:ckpt_depth_single]:
            img = auto_grad_checkpoint(block, img, vec, pe)
        for block in self.single_blocks[ckpt_depth_single:]:
            img = block(img, vec, pe)

        img = img[:, txt.shape[1] :, ...]

        img = self.final_layer(img, vec)  # (N, T, patch_size ** 2 * out_channels)
        return img

In [ ]:
# @MODELS.register_module("flux")
def Flux(
    cache_dir: str = None,
    from_pretrained: str = None,
    device_map: str | torch.device = "cuda",
    torch_dtype: torch.dtype = torch.bfloat16,
    strict_load: bool = False,
    **kwargs,
) -> MMDiTModel:
    """
    MMDiTモデルを作成するファクトリ関数
    Args:
        cache_dir (str, optional): 事前学習済みモデルのキャッシュディレクトリ. Defaults to None.
        from_pretrained (str, optional): 事前学習済みモデルのパスまたは名前. Defaults to None.
        device_map (str | torch.device, optional): モデルを配置するデバイス. Defaults to "cuda".
        torch_dtype (torch.dtype, optional): モデルのデフォルトデータ型. Defaults to torch.bfloat16.
        strict_load (bool, optional): チェックポイントの厳密な読み込みを行うかどうか. Defaults to False.
    Returns:
        MMDiTModel: 初期化されたMMDiTモデル
    """
    logger.info(f"Flux {from_pretrained=} {device_map=} {torch_dtype=} {strict_load=} {kwargs=}")

    config = MMDiTConfig(
        from_pretrained=from_pretrained,
        cache_dir=cache_dir,
        **kwargs,
    )
    low_precision_init = from_pretrained is not None and len(from_pretrained) > 0
    if low_precision_init:
        default_dtype = torch.get_default_dtype()
        torch.set_default_dtype(torch_dtype)
    with torch.device(device_map):
        model = MMDiTModel(config)
    if low_precision_init:
        torch.set_default_dtype(default_dtype)
    else:
        model = model.to(torch_dtype)
    if from_pretrained:
        model = load_checkpoint(
            model,
            from_pretrained,
            cache_dir=cache_dir,
            device_map=device_map,
            strict=strict_load,
        )
    return model


# {'type': 'flux',
#  'from_pretrained': None,
#  'strict_load': False,
#  'guidance_embed': False,
#  'fused_qkv': False,
#  'use_liger_rope': True,
#  'grad_ckpt_settings': (8, 100),
#  'in_channels': 64,
#  'vec_in_dim': 768,
#  'context_in_dim': 4096,
#  'hidden_size': 384,
#  'mlp_ratio': 4.0,
#  'num_heads': 3,
#  'depth': 1,
#  'depth_single_blocks': 38,
#  'axes_dim': [16, 56, 56],
#  'theta': 10000,
#  'qkv_bias': True}

model = Flux(
    from_pretrained=None,
    strict_load=False,
    guidance_embed=False,
    fused_qkv=False,
    use_liger_rope=True,
    grad_ckpt_settings=(8, 100),
    in_channels=64,
    vec_in_dim=768,
    context_in_dim=4096,
    hidden_size=384,
    mlp_ratio=4.0,
    num_heads=3,
    depth=1,
    depth_single_blocks=38,
    axes_dim=[16, 56, 56],
    theta=10000,
    qkv_bias=True,
)

# (batch_size, seq_len, in_channels)
sample_input = torch.randn(1, 16, 64).to("cuda").to(torch.bfloat16)

# (batch_size, seq_len)
sample_img_ids = torch.randint(0, 10000, (1, 16, 3)).to("cuda")

# (batch_size, seq_len, context_in_dim)
sample_txt = torch.randn(1, 16, 4096).to("cuda").to(torch.bfloat16)

# (batch_size, seq_len)
sample_txt_ids = torch.randint(0, 10000, (1, 16, 3)).to("cuda")

# (batch_size,)
sample_timesteps = torch.randint(0, 1000, (1,)).to("cuda").to(torch.bfloat16)

# (batch_size, vec_in_dim)
sample_y_vec = torch.randn(1, 768).to("cuda").to(torch.bfloat16)

output = model(
    img=sample_input,
    img_ids=sample_img_ids,
    txt=sample_txt,
    txt_ids=sample_txt_ids,
    timesteps=sample_timesteps,
    y_vec=sample_y_vec,
)

# (1, 16, patch_size ** 2 * out_channels)
print(output.shape)

### 訓練設定

In [ ]:
from opensora.utils.config import read_config, merge_args
import argparse
from mmengine.config import Config

def parse_args(args) -> tuple[str, argparse.Namespace]:
    """
    This function parses the command line arguments.

    Returns:
        tuple[str, argparse.Namespace]: The path to the configuration file and the command line arguments.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("config", type=str, help="model config file path")
    args, unknown_args = parser.parse_known_args(args)
    return args.config, unknown_args

def parse_configs(args) -> Config:
    """
    This function parses the configuration file and command line arguments.

    Returns:
        Config: The configuration object.
    """
    config, args = parse_args(args)
    cfg = read_config(config)
    cfg = merge_args(cfg, args)
    cfg.config_path = config

    # hard-coded for spatial compression
    if cfg.get("ae_spatial_compression", None) is not None:
        os.environ["AE_SPATIAL_COMPRESSION"] = str(cfg.ae_spatial_compression)

    return cfg

In [ ]:
# DC-AEの訓練

# https://github.com/hpcaitech/Open-Sora/blob/main/docs/hcae.md

# torchrun --nproc_per_node 8 scripts/vae/train.py configs/vae/train/video_dc_ae.py

args_1 = [
    "configs/vae/train/video_dc_ae.py",
]

# torchrun --nproc_per_node 8 scripts/vae/train.py configs/vae/train/video_dc_ae_disc.py --model.from_pretrained <model_ckpt>

args_2 = [
    "configs/vae/train/video_dc_ae_disc.py",
    "--model.from_pretrained",
    "<model_ckpt>"
]

# torchrun --nproc_per_node 8 scripts/diffusion/train.py configs/diffusion/train/stage1.py --dataset.data-path datasets/pexels_45k_necessary.csv

args_3 = [
    "configs/diffusion/train/stage1.py",
    "--dataset.data-path",
    "datasets/pexels_45k_necessary.csv"
]

# cfg = parse_configs(args_3)
# cfg.model